<a href="https://colab.research.google.com/github/seanard1/NHL_Fantasy/blob/main/Sked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime
import pytz
import scipy.stats


import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

In [9]:
baseURL = 'https://statsapi.web.nhl.com/api/v1/schedule'

start = '?startDate=2023-10-26'

end = '&endDate=2023-10-27'

In [10]:
skedURL = baseURL + start + end

sked = requests.get(skedURL).json()

In [11]:
days = sked['dates']

eastern = pytz.timezone('US/Eastern')

In [12]:
teamURL = 'https://statsapi.web.nhl.com/api/v1/teams'

teams = requests.get(teamURL).json()
teams = teams['teams']

team_names = pd.DataFrame()

for team in teams:
    abbreviation = team['abbreviation']
    name = team['name']
    id = team['id']

    temp = pd.DataFrame({'abbreviation': [abbreviation], 'name': [name], 'id': [id]})

    team_names = pd.concat([team_names, temp], ignore_index=True)

In [13]:
# See teams with most games

teams_data = {}
games_data = []

for day in days:
    games = day['games']

    for game in games:
        home_team = game['teams']['home']['team']['name']
        away_team = game['teams']['away']['team']['name']

        if home_team not in teams_data:
            teams_data[home_team] = {'played': 0, 'home': 0, 'away': 0}
        if away_team not in teams_data:
            teams_data[away_team] = {'played': 0, 'home': 0, 'away': 0}

        teams_data[home_team]['played'] += 1
        teams_data[away_team]['played'] += 1
        teams_data[home_team]['home'] += 1
        teams_data[away_team]['away'] += 1

        games_data.append(game['gamePk'])

# Create a DataFrame
teams_df = pd.DataFrame.from_dict(teams_data, orient='index')
teams_df = teams_df.sort_values(by=['played', 'home'], ascending=False)

teams_df

,played,home,away
Carolina Hurricanes,2,2,0
San Jose Sharks,2,0,2
Minnesota Wild,2,0,2
St. Louis Blues,2,0,2
Boston Bruins,1,1,0
Detroit Red Wings,1,1,0
Montréal Canadiens,1,1,0
Pittsburgh Penguins,1,1,0
Tampa Bay Lightning,1,1,0
Philadelphia Flyers,1,1,0


In [14]:
# See teams that play back-to-back

def check_consecutive_dates(game1, game2):
    date_format = '%Y-%m-%dT%H:%M:%SZ'
    date1 = datetime.strptime(game1['gameDate'], date_format)
    date2 = datetime.strptime(game2['gameDate'], date_format)
    return (date2 - date1).days == 1

teams_playing_consecutive_days = set()

for i in range(len(days)-1):
    current_day_games = days[i]['games']
    next_day_games = days[i+1]['games']

    teams_on_next_day = set()

    for game2 in next_day_games:
        home_team_2 = game2['teams']['home']['team']['name']
        away_team_2 = game2['teams']['away']['team']['name']
        teams_on_next_day.add(home_team_2)
        teams_on_next_day.add(away_team_2)

    for game1 in current_day_games:
        home_team_1 = game1['teams']['home']['team']['name']
        away_team_1 = game1['teams']['away']['team']['name']

        if (home_team_1 in teams_on_next_day) and check_consecutive_dates(game1, game2):
            teams_playing_consecutive_days.add(home_team_1)
        elif (away_team_1 in teams_on_next_day) and check_consecutive_dates(game1, game2):
            teams_playing_consecutive_days.add(away_team_1)

print("Teams playing on consecutive days:")
for team in teams_playing_consecutive_days:
    print(team)

Teams playing on consecutive days:
San Jose Sharks
Minnesota Wild
St. Louis Blues
Carolina Hurricanes


In [15]:
str(games_data[0]-1)[-2:]

'99'

In [17]:
baseURL = 'https://statsapi.web.nhl.com/api/v1/game/'
appendix = '/feed/live'

year = '202302'
game = 1
range_end = int(str(games_data[0]-1)[-2:])

awaylogs = {}
homelogs = {}


for i in range(range_end-1):

    gameURL = baseURL + year + str(i+1).zfill(4) + appendix

    print(gameURL)

    game = requests.get(gameURL).json()

    away = game['liveData']['boxscore']['teams']['away']['players']
    awayname = game['gameData']['teams']['away']['abbreviation']
    home = game['liveData']['boxscore']['teams']['home']['players']
    homename = game['gameData']['teams']['home']['abbreviation']
    birthday = game['gameData']['players']

    if game['liveData']['linescore']['currentPeriod'] > 3:
        OT = True
    else:
        OT = False

    awayteam = []

    date = game['gameData']['datetime']['dateTime']

    datetime_obj = datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ')

    utc_timestamp = pytz.utc.localize(datetime_obj)
    eastern_timezone = pytz.timezone('US/Eastern')
    eastern_time = utc_timestamp.astimezone(eastern_timezone)

    formatted_date_string = eastern_time.strftime('%d-%m-%Y')

    for player in away:
        awayteam.append(player)

    for player in awayteam:
        name = away[player]['person']['fullName']
        birth_date = birthday[player]['birthDate']
        stats = away[player]['stats']
        try:
            if list(stats.keys())[0] == 'skaterStats':
                position = away[player]['position']['abbreviation']
                stats = stats['skaterStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                stats['evenTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['evenTimeOnIce'].split(':'))))
                stats['powerPlayTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['powerPlayTimeOnIce'].split(':'))))
                stats['shortHandedTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['shortHandedTimeOnIce'].split(':'))))
            if list(stats.keys())[0] == 'goalieStats':
                position = 'G'
                stats = stats['goalieStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                if OT & (stats['decision'] == "L"):
                    stats['decision'] = 'OTL'
                shots_against = stats['shortHandedShotsAgainst'] + stats['evenShotsAgainst'] + stats['powerPlayShotsAgainst']
                if (stats['saves'] == shots_against) & ((stats['decision'] == "W") | (stats['decision'] == "OTL")):
                    stats['shutout'] = '1'
                else:
                    stats['shutout'] = '0'
        except:
            position = away[player]['position']['abbreviation']
            stats['name'] = name

        awaylogs[player  + '-' + formatted_date_string] = stats
        awaylogs[player  + '-' + formatted_date_string]['position'] = position
        awaylogs[player  + '-' + formatted_date_string]['date'] = formatted_date_string
        awaylogs[player  + '-' + formatted_date_string]['name'] = name
        awaylogs[player  + '-' + formatted_date_string]['team'] = awayname
        awaylogs[player  + '-' + formatted_date_string]['opponent'] = homename
        awaylogs[player  + '-' + formatted_date_string]['birthdate'] = birth_date
        awaylogs[player  + '-' + formatted_date_string]['id'] = player


    hometeam = []

    # date = game['gameData']['datetime']['dateTime']

    for player in home:
        hometeam.append(player)

    for player in hometeam:
        name = home[player]['person']['fullName']
        birth_date = birthday[player]['birthDate']
        stats = home[player]['stats']
        try:
            if list(stats.keys())[0] == 'skaterStats':
                position = home[player]['position']['abbreviation']
                stats = stats['skaterStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                stats['evenTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['evenTimeOnIce'].split(':'))))
                stats['powerPlayTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['powerPlayTimeOnIce'].split(':'))))
                stats['shortHandedTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['shortHandedTimeOnIce'].split(':'))))
            if list(stats.keys())[0] == 'goalieStats':
                position = 'G'
                stats = stats['goalieStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                if OT & (stats['decision'] == "L"):
                    stats['decision'] = 'OTL'
                shots_against = stats['shortHandedShotsAgainst'] + stats['evenShotsAgainst'] + stats['powerPlayShotsAgainst']
                if (stats['saves'] == shots_against) & ((stats['decision'] == "W") | (stats['decision'] == "OTL")):
                    stats['shutout'] = '1'
                else:
                    stats['shutout'] = '0'
        except:
            position = home[player]['position']['abbreviation']
            stats['name'] = name

        homelogs[player  + '-' + formatted_date_string] = stats
        homelogs[player  + '-' + formatted_date_string]['position'] = position
        homelogs[player  + '-' + formatted_date_string]['date'] = formatted_date_string
        homelogs[player  + '-' + formatted_date_string]['name'] = name
        homelogs[player  + '-' + formatted_date_string]['team'] = homename
        homelogs[player  + '-' + formatted_date_string]['opponent'] = awayname
        homelogs[player  + '-' + formatted_date_string]['birthdate'] = birth_date
        homelogs[player  + '-' + formatted_date_string]['id'] = player

#         game+=1

https://statsapi.web.nhl.com/api/v1/game/2023020001/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020002/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020003/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020004/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020005/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020006/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020007/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020008/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020009/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020010/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020011/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020012/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020013/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020014/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020015/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020016/feed/live
https://

In [18]:
home_df = pd.DataFrame(homelogs)
home_df = home_df.transpose()

columns_to_convert1 = ['timeOnIce', 'assists', 'goals', 'shots', 'hits', 'powerPlayGoals', 'powerPlayAssists', 'penaltyMinutes']
columns_to_convert2 = ['faceOffWins', 'faceoffTaken', 'takeaways', 'giveaways', 'shortHandedGoals', 'shortHandedAssists']
columns_to_convert3 = ['blocked', 'plusMinus', 'evenTimeOnIce', 'powerPlayTimeOnIce', 'shortHandedTimeOnIce', 'pim', 'shutout']
columns_to_convert4 = ['saves', 'powerPlaySaves', 'shortHandedSaves', 'evenSaves', 'shortHandedShotsAgainst', 'evenShotsAgainst', 'powerPlayShotsAgainst']

home_df[columns_to_convert1] = home_df[columns_to_convert1].fillna(0).astype(int)
home_df[columns_to_convert2] = home_df[columns_to_convert2].fillna(0).astype(int)
home_df[columns_to_convert3] = home_df[columns_to_convert3].fillna(0).astype(int)
home_df[columns_to_convert4] = home_df[columns_to_convert4].fillna(0).astype(int)


away_df = pd.DataFrame(awaylogs)
away_df = away_df.transpose()


away_df[columns_to_convert1] = away_df[columns_to_convert1].fillna(0).astype(int)
away_df[columns_to_convert2] = away_df[columns_to_convert2].fillna(0).astype(int)
away_df[columns_to_convert3] = away_df[columns_to_convert3].fillna(0).astype(int)
away_df[columns_to_convert4] = away_df[columns_to_convert4].fillna(0).astype(int)

all_df = pd.concat([home_df, away_df], axis=0)

all_df['date'] = pd.to_datetime(all_df['date'], format='%d-%m-%Y')
all_df['birthdate'] = pd.to_datetime(all_df['birthdate'], format='%Y-%m-%d')
player_list = all_df['name'].unique()
all_df['name'].nunique()
all_df = all_df.sort_values(by='date')

In [19]:
pd.set_option('display.max_columns', None)
all_df

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,position,date,name,team,opponent,birthdate,id,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage,shutout
ID8478519-10-10-2023,1002,0,0,1,0,0,0,0,60.0,9,15,1,0,0,0,0,-1,690,87,225,C,2023-10-10,Anthony Cirelli,TBL,NSH,1997-07-15,ID8478519,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
ID8477450-10-10-2023,774,1,1,3,2,0,0,0,40.0,6,15,0,0,0,0,3,1,612,18,144,C,2023-10-10,Jason Dickinson,CHI,PIT,1995-07-04,ID8477450,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
ID8476473-10-10-2023,1078,0,0,0,1,0,0,0,NaN,0,0,0,0,0,0,0,-1,971,18,89,D,2023-10-10,Connor Murphy,CHI,PIT,1993-03-26,ID8476473,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
ID8475791-10-10-2023,1244,0,0,2,1,0,0,0,NaN,0,0,1,1,0,0,0,0,945,299,0,LW,2023-10-10,Taylor Hall,CHI,PIT,1991-11-14,ID8475791,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
ID8480252-10-10-2023,555,0,1,2,0,0,0,0,54.55,6,11,0,0,0,0,0,0,531,4,20,C,2023-10-10,Cole Guttman,CHI,PIT,1999-04-06,ID8480252,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID8476392-24-10-2023,1012,1,0,1,1,0,0,0,25.0,5,20,0,0,0,0,2,0,828,2,182,C,2023-10-24,Adam Lowry,WPG,STL,1993-03-29,ID8476392,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
ID8480145-24-10-2023,1124,2,0,1,0,0,0,0,NaN,0,0,0,0,0,0,0,2,884,91,149,D,2023-10-24,Neal Pionk,WPG,STL,1995-07-29,ID8480145,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
ID8480113-24-10-2023,1066,0,0,4,0,0,0,0,NaN,0,0,1,0,0,0,0,0,864,202,0,LW,2023-10-24,Alex Iafallo,WPG,STL,1993-12-21,ID8480113,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
ID8478891-24-10-2023,912,0,1,2,0,0,0,0,NaN,0,0,1,1,0,0,0,1,799,4,109,C,2023-10-24,Mason Appleton,WPG,STL,1996-01-15,ID8478891,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0


In [20]:
all_df.loc[all_df['name'] == 'Devon Levi']

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,position,date,name,team,opponent,birthdate,id,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage,shutout
ID8482221-12-10-2023,3534,0,0,30,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-12,Devon Levi,BUF,NYR,2001-12-27,ID8482221,0,26,3,0,23,1,25,4,L,86.666667,75.0,0.0,92.0,0
ID8482221-14-10-2023,3572,0,0,29,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-14,Devon Levi,BUF,NYI,2001-12-27,ID8482221,0,26,3,0,23,0,26,3,L,89.655172,100.0,NaN,88.461538,0
ID8482221-17-10-2023,3706,0,0,25,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-17,Devon Levi,BUF,TBL,2001-12-27,ID8482221,0,23,3,0,20,0,22,3,W,92.0,100.0,NaN,90.909091,0
ID8482221-19-10-2023,3538,0,0,36,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-19,Devon Levi,BUF,CGY,2001-12-27,ID8482221,0,32,8,1,23,1,27,8,L,88.888889,100.0,100.0,85.185185,0
ID8482221-21-10-2023,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,N/A,2023-10-21,Devon Levi,BUF,NYI,2001-12-27,ID8482221,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
ID8482221-23-10-2023,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,N/A,2023-10-23,Devon Levi,BUF,MTL,2001-12-27,ID8482221,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
ID8482221-24-10-2023,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,N/A,2023-10-24,Devon Levi,BUF,OTT,2001-12-27,ID8482221,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0


In [21]:
for index, row in all_df.iterrows():
    d = 0
    so = 0
    w = 0

    if row['timeOnIce'] > 0:
      all_df.at[index, 'gamePlayed'] = 1


    if row['position'] == "G":
        if (row['shutout'] == 1) & (row['timeOnIce'] < 3446):
            all_df.at[index, 'shutout'] = 0
        if row['shutout'] == 1:
            so = 3
        else:
            so = 0
        if row['decision'] == "W":
            d = 4
            w += 1
        elif row['decision'] == "OTL":
            d = 1
        elif row['decision'] == "":
            d = 0
        else:
            d = 0
        shots_against = row['shortHandedShotsAgainst'] + row['evenShotsAgainst'] + row['powerPlayShotsAgainst']
        all_df.at[index, 'shots_against'] = shots_against
        ga = (shots_against - row['saves']) * -2
        all_df.at[index, 'goals_against'] = abs(ga)/2
        saves = row['saves'] * .2
        fp = d + so + ga + saves
        all_df.at[index, 'fantasyPoints'] = fp
        all_df.at[index, 'decisionPoints'] = d
        all_df.at[index, 'wins'] = w
#         row['fantasyPoints'] = fp
    else:
        st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
        fp = (row['goals']*2) + row['assists'] + (st * .5) + (row['blocked'] * .5) + ((row['hits'] + row['shots']) * .1)
        all_df.at[index, 'fantasyPoints'] = fp
        all_df.at[index, 'specialTeams'] = st
#         row['fantasyPoints'] = fp

In [22]:
all_df



,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,position,date,name,team,opponent,birthdate,id,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage,shutout,gamePlayed,fantasyPoints,specialTeams,shots_against,goals_against,decisionPoints,wins
ID8478519-10-10-2023,1002,0,0,1,0,0,0,0,60.0,9,15,1,0,0,0,0,-1,690,87,225,C,2023-10-10,Anthony Cirelli,TBL,NSH,1997-07-15,ID8478519,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.1,0.0,NaN,NaN,NaN,NaN
ID8477450-10-10-2023,774,1,1,3,2,0,0,0,40.0,6,15,0,0,0,0,3,1,612,18,144,C,2023-10-10,Jason Dickinson,CHI,PIT,1995-07-04,ID8477450,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,5.0,0.0,NaN,NaN,NaN,NaN
ID8476473-10-10-2023,1078,0,0,0,1,0,0,0,NaN,0,0,0,0,0,0,0,-1,971,18,89,D,2023-10-10,Connor Murphy,CHI,PIT,1993-03-26,ID8476473,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.1,0.0,NaN,NaN,NaN,NaN
ID8475791-10-10-2023,1244,0,0,2,1,0,0,0,NaN,0,0,1,1,0,0,0,0,945,299,0,LW,2023-10-10,Taylor Hall,CHI,PIT,1991-11-14,ID8475791,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.3,0.0,NaN,NaN,NaN,NaN
ID8480252-10-10-2023,555,0,1,2,0,0,0,0,54.55,6,11,0,0,0,0,0,0,531,4,20,C,2023-10-10,Cole Guttman,CHI,PIT,1999-04-06,ID8480252,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.2,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID8476392-24-10-2023,1012,1,0,1,1,0,0,0,25.0,5,20,0,0,0,0,2,0,828,2,182,C,2023-10-24,Adam Lowry,WPG,STL,1993-03-29,ID8476392,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.2,0.0,NaN,NaN,NaN,NaN
ID8480145-24-10-2023,1124,2,0,1,0,0,0,0,NaN,0,0,0,0,0,0,0,2,884,91,149,D,2023-10-24,Neal Pionk,WPG,STL,1995-07-29,ID8480145,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.1,0.0,NaN,NaN,NaN,NaN
ID8480113-24-10-2023,1066,0,0,4,0,0,0,0,NaN,0,0,1,0,0,0,0,0,864,202,0,LW,2023-10-24,Alex Iafallo,WPG,STL,1993-12-21,ID8480113,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.4,0.0,NaN,NaN,NaN,NaN
ID8478891-24-10-2023,912,0,1,2,0,0,0,0,NaN,0,0,1,1,0,0,0,1,799,4,109,C,2023-10-24,Mason Appleton,WPG,STL,1996-01-15,ID8478891,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.2,0.0,NaN,NaN,NaN,NaN


In [23]:
# Build summary stats for rankings

def first_non_na_position(x):
    non_na_values = x[x != 'N/A']
    return non_na_values.iloc[0] if not non_na_values.empty else 'N/A'

summary_stats = all_df.groupby('id').agg({
    'name': 'first',
    'team': 'first',
    'position': first_non_na_position,
    'timeOnIce': 'sum',
    'gamePlayed': 'sum',
    'goals': 'sum',
    'assists': 'sum',
    'specialTeams': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
    'evenTimeOnIce': 'sum',
    'powerPlayTimeOnIce': 'sum',
    'shortHandedTimeOnIce': 'sum',
    'saves': 'sum',
    'powerPlaySaves': 'sum',
    'shortHandedSaves': 'sum',
    'evenSaves': 'sum',
    'shortHandedShotsAgainst': 'sum',
    'evenShotsAgainst': 'sum',
    'powerPlayShotsAgainst': 'sum',
    'decisionPoints': 'sum',
    'wins': 'sum',
    # 'decision': 'sum',
    'shutout': 'sum',
    'fantasyPoints': 'sum'
}).reset_index()


# summary_stats = summary_stats.rename(columns={'date': 'gp'})
# summary_stats['gp'] = summary_stats['gp'].apply(lambda x: len(x))
summary_stats['gamePlayed'] = summary_stats['gamePlayed'].astype(int)
summary_stats['decisionPoints'] = summary_stats['decisionPoints'].astype(int)
summary_stats['wins'] = summary_stats['wins'].astype(int)
summary_stats['specialTeams'] = summary_stats['specialTeams'].astype(int)


summary_stats['FPP60'] = (summary_stats['fantasyPoints'] / summary_stats['timeOnIce'] * 3600).round(2)
summary_stats['FPPG'] = (summary_stats['fantasyPoints'] / summary_stats['gamePlayed']).round(2)

summary_stats = summary_stats.sort_values(by='FPPG', ascending=False)
# pd.set_option('display.max_rows', None)
summary_stats.set_index('name', inplace=True)

summary_stats.head()

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,blocked,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG
name,,,,,,,,,,,,,,,,,,,,,,,,,,,
James Reimer,ID8473503,DET,G,7200,2,0,0,0,54,0,0,0,0,0,52,10,1,41,1,43,10,8,2,1,17.4,8.70,8.70
Jeremy Swayman,ID8480280,BOS,G,10730,3,0,0,0,93,0,0,0,0,0,89,23,4,62,4,65,24,12,3,1,24.8,8.32,8.27
Anthony Stolarz,ID8476932,FLA,G,3569,1,0,0,0,28,0,0,0,0,0,27,2,2,23,2,23,3,4,1,0,7.4,7.46,7.40
Linus Ullmark,ID8476999,BOS,G,10793,3,0,0,0,79,0,0,0,0,0,76,6,6,64,6,67,6,12,3,0,21.2,7.07,7.07
Joseph Woll,ID8479361,TOR,G,9982,3,0,0,0,94,0,0,0,0,0,90,19,1,70,1,72,21,8,2,0,21.0,7.57,7.00


In [24]:
summary_stats.loc[summary_stats['team'] == 'BUF']

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,blocked,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG
name,,,,,,,,,,,,,,,,,,,,,,,,,,,
Ukko-Pekka Luukkonen,ID8480045,BUF,G,3582,1,0,0,0,38,0,0,0,0,0,34,8,0,26,0,29,9,4,1,0,2.8,2.81,2.80
Eric Comrie,ID8477480,BUF,G,7013,2,0,0,0,52,0,0,0,0,0,48,11,3,34,3,37,12,4,1,0,5.6,2.87,2.80
Rasmus Dahlin,ID8480839,BUF,D,10173,7,0,7,3,18,16,12,8615,1415,143,0,0,0,0,0,0,0,0,0,0,17.9,6.33,2.56
Jeff Skinner,ID8475784,BUF,LW,7456,7,5,1,1,25,0,2,5979,1436,41,0,0,0,0,0,0,0,0,0,0,15.0,7.24,2.14
Tage Thompson,ID8479420,BUF,C,7890,7,3,1,3,32,5,1,5441,1416,1033,0,0,0,0,0,0,0,0,0,0,12.7,5.79,1.81
Dylan Cozens,ID8481528,BUF,C,6848,7,2,4,2,18,9,1,5323,1441,84,0,0,0,0,0,0,0,0,0,0,12.2,6.41,1.74
Mattias Samuelsson,ID8480807,BUF,D,8386,7,1,1,0,5,15,13,6772,9,1605,0,0,0,0,0,0,0,0,0,0,11.5,4.94,1.64
Erik Johnson,ID8473446,BUF,D,6343,7,1,0,0,7,10,13,4669,0,1674,0,0,0,0,0,0,0,0,0,0,10.2,5.79,1.46
Casey Mittelstadt,ID8479999,BUF,C,6988,7,1,5,0,7,0,3,6250,730,8,0,0,0,0,0,0,0,0,0,0,9.2,4.74,1.31


In [25]:
# https://statsapi.web.nhl.com/api/v1/people/8477953/stats?stats=gameLog&season=20232024
# This is a game log endpoint

goalies = all_df.loc[all_df['position'] == 'G']

skaters = all_df.loc[all_df['position'] != 'G']
skaters['specialTeams'] = skaters['specialTeams'].astype(int)
skaters

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,position,date,name,team,opponent,birthdate,id,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage,shutout,gamePlayed,fantasyPoints,specialTeams,shots_against,goals_against,decisionPoints,wins
ID8478519-10-10-2023,1002,0,0,1,0,0,0,0,60.0,9,15,1,0,0,0,0,-1,690,87,225,C,2023-10-10,Anthony Cirelli,TBL,NSH,1997-07-15,ID8478519,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.1,0,NaN,NaN,NaN,NaN
ID8477450-10-10-2023,774,1,1,3,2,0,0,0,40.0,6,15,0,0,0,0,3,1,612,18,144,C,2023-10-10,Jason Dickinson,CHI,PIT,1995-07-04,ID8477450,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,5.0,0,NaN,NaN,NaN,NaN
ID8476473-10-10-2023,1078,0,0,0,1,0,0,0,NaN,0,0,0,0,0,0,0,-1,971,18,89,D,2023-10-10,Connor Murphy,CHI,PIT,1993-03-26,ID8476473,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.1,0,NaN,NaN,NaN,NaN
ID8475791-10-10-2023,1244,0,0,2,1,0,0,0,NaN,0,0,1,1,0,0,0,0,945,299,0,LW,2023-10-10,Taylor Hall,CHI,PIT,1991-11-14,ID8475791,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.3,0,NaN,NaN,NaN,NaN
ID8480252-10-10-2023,555,0,1,2,0,0,0,0,54.55,6,11,0,0,0,0,0,0,531,4,20,C,2023-10-10,Cole Guttman,CHI,PIT,1999-04-06,ID8480252,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.2,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID8476392-24-10-2023,1012,1,0,1,1,0,0,0,25.0,5,20,0,0,0,0,2,0,828,2,182,C,2023-10-24,Adam Lowry,WPG,STL,1993-03-29,ID8476392,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.2,0,NaN,NaN,NaN,NaN
ID8480145-24-10-2023,1124,2,0,1,0,0,0,0,NaN,0,0,0,0,0,0,0,2,884,91,149,D,2023-10-24,Neal Pionk,WPG,STL,1995-07-29,ID8480145,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.1,0,NaN,NaN,NaN,NaN
ID8480113-24-10-2023,1066,0,0,4,0,0,0,0,NaN,0,0,1,0,0,0,0,0,864,202,0,LW,2023-10-24,Alex Iafallo,WPG,STL,1993-12-21,ID8480113,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.4,0,NaN,NaN,NaN,NaN
ID8478891-24-10-2023,912,0,1,2,0,0,0,0,NaN,0,0,1,1,0,0,0,1,799,4,109,C,2023-10-24,Mason Appleton,WPG,STL,1996-01-15,ID8478891,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.2,0,NaN,NaN,NaN,NaN


In [26]:
opponent_stats = skaters.groupby('opponent').agg({
    'timeOnIce': 'sum',
    'assists': 'sum',
    'goals': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
#     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
    'fantasyPoints': 'sum',
    'date': 'nunique',
    'powerPlayGoals': 'sum',
    'powerPlayAssists': 'sum'
    # Add more columns as needed
})

opponent_stats['FPAPG'] = round(opponent_stats['fantasyPoints'] / opponent_stats['date'], 2)
opponent_stats['FPAPG_Rank'] = opponent_stats['FPAPG'].rank(ascending=True, method='min').astype(int)

# opponent_stats['BPG'] = round(opponent_stats['blocked'] / opponent_stats['date'], 2)
# opponent_stats['BPG_Rank'] = opponent_stats['BPG'].rank(ascending=True, method='min').astype(int)

opponent_stats = opponent_stats.rename_axis('abbreviation')

opponent_stats = pd.merge(opponent_stats, team_names, on='abbreviation')

opponent_stats['powerPlayPoints'] = opponent_stats['powerPlayGoals'] + opponent_stats['powerPlayAssists']

opponent_stats = opponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists'])

opponent_stats

,abbreviation,timeOnIce,assists,goals,shots,hits,blocked,fantasyPoints,date,FPAPG,FPAPG_Rank,name,id,powerPlayPoints
0,ANA,105773,33,17,187,84,85,148.6,6,24.77,14,Anaheim Ducks,24,24
1,ARI,105834,28,15,185,134,82,139.9,6,23.32,10,Arizona Coyotes,53,18
2,BOS,106062,13,7,172,110,96,104.7,6,17.45,1,Boston Bruins,6,3
3,BUF,124006,40,22,211,157,114,183.8,7,26.26,18,Buffalo Sabres,7,9
4,CAR,123121,54,33,187,99,140,235.1,7,33.59,32,Carolina Hurricanes,12,27
5,CBJ,106739,30,19,199,67,104,152.1,6,25.35,15,Columbus Blue Jackets,29,9
6,CGY,124356,43,25,195,89,119,185.4,7,26.49,20,Calgary Flames,20,9
7,CHI,122819,38,21,254,115,109,178.4,7,25.49,16,Chicago Blackhawks,16,9
8,COL,107131,19,12,172,100,97,121.7,6,20.28,2,Colorado Avalanche,21,6
9,DAL,90676,17,10,166,115,74,105.6,5,21.12,4,Dallas Stars,25,0


In [27]:
skaters_temp = skaters
# replace_dict = {'LW': 'W', 'RW': 'W'}
skaters_temp['position'] = skaters_temp['position'].replace('LW', 'W')
skaters_temp['position'] = skaters_temp['position'].replace('RW', 'W')
skaters_temp = skaters_temp.loc[skaters_temp['gamePlayed']>0]


opponent_stats_by_pos = skaters_temp.groupby(['opponent', 'position']).agg({
    'timeOnIce': 'sum',
    'assists': 'sum',
    'goals': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
    'fantasyPoints': 'sum',
    'date': 'nunique',
    'powerPlayGoals': 'sum',
    'powerPlayAssists': 'sum',
    'team': 'unique'
    # Add more columns as needed
})
opponent_stats_by_pos

opponent_stats_by_pos['powerPlayPoints'] = opponent_stats_by_pos['powerPlayGoals'] + opponent_stats_by_pos['powerPlayAssists']

opponent_stats_by_pos = opponent_stats_by_pos.drop(columns=['powerPlayGoals', 'powerPlayAssists'])

opponent_stats_by_pos = opponent_stats_by_pos.rename(columns={'date': 'gamesPlayed'})
opponent_stats_by_pos

timeOnIce  assists  goals  shots  hits  blocked  \
opponent position                                                    
ANA      C             39307       13      8     69    27       21   
         D             40717       16      2     56    33       46   
         W             25749        4      7     62    24       18   
ARI      C             31037        9      5     68    43       17   
         D             41820        9      1     48    41       47   
...                      ...      ...    ...    ...   ...      ...   
WPG      D             41135        7      5     50    42       58   
         W             26661       10      7     53    33       19   
WSH      C             27013       14     10     54    19       11   
         D             34724       10      3     40    18       55   
         W             27452       11      6     55    23       14   

                   fantasyPoints  gamesPlayed                            team  \
opponent position                                                               
ANA      C                  53.6            6  [VGK, CAR, DAL, ARI, BOS, CBJ]   
         D                  56.4            6  [VGK, CAR, DAL, ARI, BOS, CBJ]   
         W                  38.6            6  [VGK, CAR, DAL, ARI, BOS, CBJ]   
ARI      C                  42.1            6  [NJD, NYR, NYI, STL, ANA, LAK]   
         D                  44.9            6  [NJD, NYR, NYI, STL, ANA, LAK]   
...                          ...          ...                             ...   
WPG      D                  57.7            6  [CGY, FLA, LAK, VGK, EDM, STL]   
         W                  44.6            6  [CGY, FLA, LAK, VGK, EDM, STL]   
WSH      C                  52.8            5       [PIT, CGY, OTT, MTL, TOR]   
         D                  51.3            5       [PIT, CGY, OTT, MTL, TOR]   
         W                  40.3            5       [PIT, CGY, OTT, MTL, TOR]   

                   powerPlayPoints  
opponent position                   
ANA      C                       9  
         D                       9  
         W                       6  
ARI      C                       7  
         D                       3  
...                            ...  
WPG      D                       4  
         W                       5  
WSH      C                      12  
         D                       4  
         W                       5  

[96 rows x 10 columns]

In [28]:
D = opponent_stats_by_pos.xs('D', level='position')
D['position'] = 'D'


D['FPAPG'] = round(D['fantasyPoints'] / D['gamesPlayed'], 2)
D['FPAPG_Rank'] = D['FPAPG'].rank(ascending=True, method='min').astype(int)

# D['SPG'] = round(D['shots'] / D['gamesPlayed'], 2)
# D['SPG_Rank'] = D['SPG'].rank(ascending=True, method='min').astype(int)

# D['BPG'] = round(D['shots'] / D['gamesPlayed'], 2)
# D['BPG_Rank'] = D['BPG'].rank(ascending=True, method='min').astype(int)

# D['PPG'] = round((D['assists'] + D['goals']) / D['gamesPlayed'], 2)
# D['PPG_Rank'] = D['PPG'].rank(ascending=True, method='min').astype(int)

D

,timeOnIce,assists,goals,shots,hits,blocked,fantasyPoints,gamesPlayed,team,powerPlayPoints,position,FPAPG,FPAPG_Rank
opponent,,,,,,,,,,,,,
ANA,40717,16,2,56,33,46,56.4,6,"[VGK, CAR, DAL, ARI, BOS, CBJ]",9,D,9.40,18
ARI,41820,9,1,48,41,47,44.9,6,"[NJD, NYR, NYI, STL, ANA, LAK]",3,D,7.48,1
BOS,40561,3,0,52,48,65,46.0,6,"[CHI, NSH, SJS, LAK, ANA]",1,D,7.67,2
BUF,48566,11,4,52,39,79,68.6,7,"[NYR, NYI, TBL, CGY, MTL, OTT]",1,D,9.80,21
CAR,48054,16,5,41,42,73,74.3,7,"[OTT, LAK, ANA, SJS, SEA, COL, TBL]",6,D,10.61,30
CBJ,42306,10,4,55,23,66,60.3,6,"[PHI, NYR, DET, CGY, MIN, ANA]",3,D,10.05,22
CGY,48968,20,4,56,40,73,75.6,7,"[WPG, PIT, WSH, BUF, CBJ, DET, NYR]",3,D,10.80,31
CHI,48622,11,1,71,35,77,63.6,7,"[PIT, BOS, MTL, TOR, COL, VGK]",2,D,9.09,17
COL,40845,6,2,42,44,59,49.6,6,"[LAK, SJS, SEA, CHI, CAR, NYI]",3,D,8.27,8


In [29]:
C = opponent_stats_by_pos.xs('C', level='position')
C['position'] = 'C'

C['FPAPG'] = round(C['fantasyPoints'] / C['gamesPlayed'], 2)
C['FPAPG_Rank'] = C['FPAPG'].rank(ascending=True, method='min').astype(int)

# C['SPG'] = round(C['shots'] / C['gamesPlayed'], 2)
# C['SPG_Rank'] = C['SPG'].rank(ascending=True, method='min').astype(int)

# C['PPPG'] = round(C['powerPlayPoints'] / C['gamesPlayed'], 2)
# C['PPPG_Rank'] = C['PPPG'].rank(ascending=True, method='min').astype(int)

# C['PPG'] = round((C['assists'] + C['goals']) / C['gamesPlayed'], 2)
# C['PPG_Rank'] = C['PPG'].rank(ascending=True, method='min').astype(int)

C

,timeOnIce,assists,goals,shots,hits,blocked,fantasyPoints,gamesPlayed,team,powerPlayPoints,position,FPAPG,FPAPG_Rank
opponent,,,,,,,,,,,,,
ANA,39307,13,8,69,27,21,53.6,6,"[VGK, CAR, DAL, ARI, BOS, CBJ]",9,C,8.93,21
ARI,31037,9,5,68,43,17,42.1,6,"[NJD, NYR, NYI, STL, ANA, LAK]",7,C,7.02,11
BOS,34845,5,4,58,36,19,32.4,6,"[CHI, NSH, SJS, LAK, ANA]",1,C,5.40,1
BUF,39229,17,7,87,40,23,57.7,7,"[NYR, NYI, TBL, CGY, MTL, OTT]",4,C,8.24,17
CAR,33472,15,10,55,21,35,64.1,7,"[OTT, LAK, ANA, SJS, SEA, COL, TBL]",7,C,9.16,23
CBJ,30784,10,5,64,14,21,40.3,6,"[PHI, NYR, DET, CGY, MIN, ANA]",2,C,6.72,8
CGY,36780,9,7,73,20,23,45.3,7,"[WPG, PIT, WSH, BUF, CBJ, DET, NYR]",3,C,6.47,7
CHI,37786,10,12,94,48,20,61.2,7,"[PIT, BOS, MTL, TOR, COL, VGK]",4,C,8.74,19
COL,31893,10,3,50,20,17,32.5,6,"[LAK, SJS, SEA, CHI, CAR, NYI]",2,C,5.42,2


In [30]:
W = opponent_stats_by_pos.xs('W', level='position')
W['position'] = 'W'

W['FPAPG'] = round(W['fantasyPoints'] / W['gamesPlayed'], 2)
W['FPAPG_Rank'] = W['FPAPG'].rank(ascending=True, method='min').astype(int)

# W['PPPG'] = round((W['powerPlayPoints']) / W['gamesPlayed'], 2)
# W['PPPG_Rank'] = W['PPPG'].rank(ascending=True, method='min').astype(int)

# W['PPG'] = round((W['assists'] + W['goals']) / W['gamesPlayed'], 2)
# W['PPG_Rank'] = W['PPG'].rank(ascending=True, method='min').astype(int)

# W['SPG'] = round((W['shots']) / W['gamesPlayed'], 2)
# W['SPG_Rank'] = W['SPG'].rank(ascending=True, method='min').astype(int)

W

,timeOnIce,assists,goals,shots,hits,blocked,fantasyPoints,gamesPlayed,team,powerPlayPoints,position,FPAPG,FPAPG_Rank
opponent,,,,,,,,,,,,,
ANA,25749,4,7,62,24,18,38.6,6,"[VGK, CAR, DAL, ARI, BOS, CBJ]",6,W,6.43,5
ARI,32977,10,9,69,50,18,52.9,6,"[NJD, NYR, NYI, STL, ANA, LAK]",8,W,8.82,25
BOS,30656,5,3,62,26,12,26.3,6,"[CHI, NSH, SJS, LAK, ANA]",1,W,4.38,1
BUF,36211,12,11,72,78,12,57.5,7,"[NYR, NYI, TBL, CGY, MTL, OTT]",4,W,8.21,17
CAR,41595,23,18,91,36,32,96.7,7,"[OTT, LAK, ANA, SJS, SEA, COL, TBL]",14,W,13.81,32
CBJ,33649,10,10,80,30,17,51.5,6,"[PHI, NYR, DET, CGY, MIN, ANA]",4,W,8.58,21
CGY,38608,14,14,66,29,23,64.5,7,"[WPG, PIT, WSH, BUF, CBJ, DET, NYR]",3,W,9.21,28
CHI,36411,17,8,89,32,12,53.6,7,"[PIT, BOS, MTL, TOR, COL, VGK]",3,W,7.66,14
COL,34393,3,7,80,36,21,39.6,6,"[LAK, SJS, SEA, CHI, CAR, NYI]",1,W,6.60,6


In [31]:
goalies

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,position,date,name,team,opponent,birthdate,id,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage,shutout,gamePlayed,fantasyPoints,specialTeams,shots_against,goals_against,decisionPoints,wins
ID8475831-10-10-2023,3577,0,0,27,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-10,Philipp Grubauer,SEA,VGK,1991-11-25,ID8475831,0,24,3,0,21,0,24,3,L,88.888889,100.0,NaN,87.5,0,1.0,-1.2,NaN,27.0,3.0,0.0,0.0
ID8477424-10-10-2023,3436,0,0,33,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-10,Juuse Saros,NSH,TBL,1995-04-19,ID8477424,0,29,8,0,21,0,23,10,L,87.878788,80.0,NaN,91.304348,0,1.0,-2.2,NaN,33.0,4.0,0.0,0.0
ID8475852-10-10-2023,3600,0,0,41,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-10,Petr Mrazek,CHI,PIT,1992-02-14,ID8475852,0,39,4,0,35,0,37,4,W,95.121951,100.0,NaN,94.594595,0,1.0,7.8,NaN,41.0,2.0,4.0,1.0
ID8478499-10-10-2023,3600,0,0,33,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-10,Adin Hill,VGK,SEA,1996-05-11,ID8478499,0,32,10,0,22,0,23,10,W,96.969697,100.0,NaN,95.652174,0,1.0,8.4,NaN,33.0,1.0,4.0,1.0
ID8477465-10-10-2023,3450,0,0,35,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-10,Tristan Jarry,PIT,CHI,1995-04-29,ID8477465,0,32,5,1,26,1,29,5,L,91.428571,100.0,100.0,89.655172,0,1.0,0.4,NaN,35.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID8477831-24-10-2023,3592,0,0,27,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-24,Pheonix Copley,LAK,ARI,1992-01-18,ID8477831,0,24,5,0,19,0,21,6,W,88.888889,83.333333,NaN,90.47619,0,1.0,2.8,NaN,27.0,3.0,4.0,1.0
ID8475852-24-10-2023,3566,0,0,43,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-24,Petr Mrazek,CHI,BOS,1992-02-14,ID8475852,0,40,4,0,36,0,39,4,L,93.023256,100.0,NaN,92.307692,0,1.0,2.0,NaN,43.0,3.0,0.0,0.0
ID8478024-24-10-2023,3895,0,0,38,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-24,Ville Husso,DET,SEA,1995-02-06,ID8478024,0,33,3,1,29,1,32,5,OTL,86.842105,60.0,100.0,90.625,0,1.0,-2.4,NaN,38.0,5.0,1.0,0.0
ID8476945-24-10-2023,3572,0,0,20,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-24,Connor Hellebuyck,WPG,STL,1993-05-19,ID8476945,0,18,2,0,16,0,17,3,W,90.0,66.666667,NaN,94.117647,0,1.0,3.6,NaN,20.0,2.0,4.0,1.0


In [33]:
gopponent_stats = goalies.groupby('opponent').agg({
    'timeOnIce': 'sum',
    'goals_against': 'sum',
    'shots': 'sum',
#     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
    'fantasyPoints': 'sum',
    'date': 'nunique',
    'powerPlayGoals': 'sum',
    'powerPlayAssists': 'sum',
#     'team': 'unique',
    'powerPlaySaves': 'sum',
    'powerPlayShotsAgainst': 'sum'
    # Add more columns as needed
})

gopponent_stats['FPAPG'] = round(gopponent_stats['fantasyPoints'] / gopponent_stats['date'], 2)
gopponent_stats['FPAPG_Rank'] = gopponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

# gopponent_stats['SPG'] = round(gopponent_stats['shots'] / gopponent_stats['date'], 2)
# gopponent_stats['SPG_Rank'] = gopponent_stats['SPG'].rank(ascending=False, method='min').astype(int)

gopponent_stats = gopponent_stats.rename_axis('abbreviation')

gopponent_stats = pd.merge(gopponent_stats, team_names, on='abbreviation')

# gopponent_stats['powerPlayPoints'] = gopponent_stats['powerPlayGoals'] + gopponent_stats['powerPlayAssists']

gopponent_stats = gopponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists', 'id'])

gopponent_stats

,abbreviation,timeOnIce,goals_against,shots,fantasyPoints,date,powerPlaySaves,powerPlayShotsAgainst,FPAPG,FPAPG_Rank,name
0,ANA,21526,12.0,166,23.8,6,28,29,3.97,6,Anaheim Ducks
1,ARI,21737,15.0,170,17.0,6,28,36,2.83,13,Arizona Coyotes
2,BOS,20963,17.0,191,0.8,6,31,34,0.13,25,Boston Bruins
3,BUF,25131,18.0,213,20.0,7,33,36,2.86,11,Buffalo Sabres
4,CAR,25395,27.0,233,7.2,7,39,47,1.03,20,Carolina Hurricanes
5,CBJ,21594,16.0,196,20.0,6,37,40,3.33,10,Columbus Blue Jackets
6,CGY,25300,16.0,226,30.0,7,36,39,4.29,5,Calgary Flames
7,CHI,24772,12.0,181,35.8,7,31,33,5.11,3,Chicago Blackhawks
8,COL,21602,23.0,222,-5.2,6,33,39,-0.87,28,Colorado Avalanche
9,DAL,18544,15.0,143,1.6,5,10,12,0.32,24,Dallas Stars


In [34]:
top_players_by_team = summary_stats.groupby('team').apply(lambda x: x.nlargest(5, 'FPPG'))
top_players_by_team.reset_index(level='team', drop=True, inplace=True)
top_players_by_team.loc[top_players_by_team['team'] == 'ANA']

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,blocked,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG
name,,,,,,,,,,,,,,,,,,,,,,,,,,,
Lukas Dostal,ID8480843,ANA,G,10791,3,0,0,0,99,0,0,0,0,0,92,29,0,63,0,66,33,8,2,0,12.4,4.14,4.13
Frank Vatrano,ID8478366,ANA,RW,6941,6,5,1,1,24,12,9,4997,748,1196,0,0,0,0,0,0,0,0,0,0,19.6,10.17,3.27
Ilya Lyubushkin,ID8480950,ANA,D,6183,6,0,0,0,5,13,21,4887,0,1296,0,0,0,0,0,0,0,0,0,0,12.3,7.16,2.05
Ryan Strome,ID8476458,ANA,C,6635,6,1,4,0,10,5,6,5338,1199,98,0,0,0,0,0,0,0,0,0,0,10.5,5.70,1.75
Leo Carlsson,ID8484153,ANA,C,2433,2,1,0,0,6,2,1,1894,516,23,0,0,0,0,0,0,0,0,0,0,3.3,4.88,1.65


In [35]:
eastern = pytz.timezone('US/Eastern')

for day in days:
    date = day['date']
    games = day['games']

    day_of_week = datetime.strptime(date, '%Y-%m-%d').strftime('%A').upper()
    print(day_of_week)

    for game in games:
        home_team = game['teams']['home']['team']['name']
        away_team = game['teams']['away']['team']['name']
        venue = game['venue']['name']
        game_time_utc = datetime.fromisoformat(game['gameDate'][:-1]).replace(tzinfo=pytz.utc)
        game_time_et = game_time_utc.astimezone(eastern)

        formatted_time = game_time_et.strftime('%I:%M %p').lstrip('0').replace(':00', '').replace('PM', 'p.m.')

        home_abbr = opponent_stats.loc[opponent_stats['name'] == home_team, 'abbreviation'].values[0]
        home_top5 = top_players_by_team.loc[top_players_by_team['team'] == home_abbr]

        away_abbr = opponent_stats.loc[opponent_stats['name'] == away_team, 'abbreviation'].values[0]
        away_top5 = top_players_by_team.loc[top_players_by_team['team'] == away_abbr]

#         print(f"On {date}, {away_team} at {home_team}")
        print(f"\033[1m{away_team} at {home_team}\033[0m")
        print(f"{formatted_time}, {venue}")
        print()
        print(f"{away_abbr}: {opponent_stats.loc[opponent_stats['name'] == away_team, 'FPAPG'].values[0]} FPAPG, Rank: {opponent_stats.loc[opponent_stats['name'] == away_team, 'FPAPG_Rank'].values[0]}")
        print(f"\033[1mTop Players in FPPG\033[0m")
        for index, row in away_top5.iterrows():
            print(f"{index}, FPPG: {row['FPPG']}, FP: {round(row['fantasyPoints'], 2)}")
        print()
        print(f"{home_abbr}: {opponent_stats.loc[opponent_stats['name'] == home_team, 'FPAPG'].values[0]} FPAPG, Rank: {opponent_stats.loc[opponent_stats['name'] == home_team, 'FPAPG_Rank'].values[0]}")
        print(f"\033[1mTop Players in FPPG\033[0m")
        for index, row in home_top5.iterrows():
            print(f"{index}, FPPG: {row['FPPG']}, FP: {round(row['fantasyPoints'], 2)}")

        print()
#         print("\033[1mThis is bold text\033[0m")


THURSDAY
Anaheim Ducks at Boston Bruins
7 p.m., TD Garden

ANA: 24.77 FPAPG, Rank: 14
Top Players in FPPG
Lukas Dostal, FPPG: 4.13, FP: 12.4
Frank Vatrano, FPPG: 3.27, FP: 19.6
Ilya Lyubushkin, FPPG: 2.05, FP: 12.3
Ryan Strome, FPPG: 1.75, FP: 10.5
Leo Carlsson, FPPG: 1.65, FP: 3.3

BOS: 17.45 FPAPG, Rank: 1
Top Players in FPPG
Jeremy Swayman, FPPG: 8.27, FP: 24.8
Linus Ullmark, FPPG: 7.07, FP: 21.2
David Pastrnak, FPPG: 2.93, FP: 17.6
Brad Marchand, FPPG: 2.82, FP: 16.9
Charlie McAvoy, FPPG: 2.22, FP: 13.3

Seattle Kraken at Carolina Hurricanes
7 p.m., PNC Arena

SEA: 27.36 FPAPG, Rank: 24
Top Players in FPPG
Joey Daccord, FPPG: 2.27, FP: 6.8
Jaden Schwartz, FPPG: 2.01, FP: 14.1
Vince Dunn, FPPG: 1.94, FP: 13.6
Jamie Oleksiak, FPPG: 1.79, FP: 12.5
Adam Larsson, FPPG: 1.74, FP: 12.2

CAR: 33.59 FPAPG, Rank: 32
Top Players in FPPG
Jaccob Slavin, FPPG: 2.83, FP: 19.8
Seth Jarvis, FPPG: 2.37, FP: 16.6
Brett Pesce, FPPG: 2.24, FP: 11.2
Jesperi Kotkaniemi, FPPG: 2.14, FP: 15.0
Brady Skjei, 

In [36]:
team_check = opponent_stats['name'].values

teams_df.index.values

for team in team_check:
    if team not in teams_df.index.values:
        print(team)

Florida Panthers
Nashville Predators


In [37]:
summary_stats = summary_stats.dropna()

In [38]:
# summary_stats = summary_stats[summary_stats['position'] != 'N/A']
summary_stats

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,blocked,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG
name,,,,,,,,,,,,,,,,,,,,,,,,,,,
James Reimer,ID8473503,DET,G,7200,2,0,0,0,54,0,0,0,0,0,52,10,1,41,1,43,10,8,2,1,17.4,8.70,8.70
Jeremy Swayman,ID8480280,BOS,G,10730,3,0,0,0,93,0,0,0,0,0,89,23,4,62,4,65,24,12,3,1,24.8,8.32,8.27
Anthony Stolarz,ID8476932,FLA,G,3569,1,0,0,0,28,0,0,0,0,0,27,2,2,23,2,23,3,4,1,0,7.4,7.46,7.40
Linus Ullmark,ID8476999,BOS,G,10793,3,0,0,0,79,0,0,0,0,0,76,6,6,64,6,67,6,12,3,0,21.2,7.07,7.07
Joseph Woll,ID8479361,TOR,G,9982,3,0,0,0,94,0,0,0,0,0,90,19,1,70,1,72,21,8,2,0,21.0,7.57,7.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pyotr Kochetkov,ID8481611,CAR,G,9151,3,0,0,0,67,0,0,0,0,0,56,8,2,46,3,52,12,0,0,0,-10.8,-4.25,-3.60
Laurent Brossoit,ID8476316,WPG,G,3564,1,0,0,0,26,0,0,0,0,0,22,2,0,20,0,22,4,0,0,0,-3.6,-3.64,-3.60
Stuart Skinner,ID8479973,EDM,G,9232,3,0,0,0,59,0,0,0,0,0,48,11,2,35,3,41,15,1,0,0,-11.4,-4.45,-3.80


In [39]:
skaters_summary = summary_stats.loc[summary_stats['position'] != "G"]
skaters_summary = skaters_summary.drop(columns=['saves', 'powerPlaySaves',	'shortHandedSaves',	'evenSaves',	'shortHandedShotsAgainst',	'evenShotsAgainst',	'powerPlayShotsAgainst',	'decisionPoints',	'wins',	'shutout'])
# skaters_summary.loc[skaters_summary.index == 'Sebastian Aho']


In [40]:
goalies_summary = summary_stats.loc[summary_stats['position'] == "G"]
goalies_summary = goalies_summary.drop(columns=['goals',	'assists',	'specialTeams',	'hits',	'blocked',	'evenTimeOnIce',	'powerPlayTimeOnIce',	'shortHandedTimeOnIce'])
goalies_summary

,id,team,position,timeOnIce,gamePlayed,shots,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG
name,,,,,,,,,,,,,,,,,,,
James Reimer,ID8473503,DET,G,7200,2,54,52,10,1,41,1,43,10,8,2,1,17.4,8.70,8.70
Jeremy Swayman,ID8480280,BOS,G,10730,3,93,89,23,4,62,4,65,24,12,3,1,24.8,8.32,8.27
Anthony Stolarz,ID8476932,FLA,G,3569,1,28,27,2,2,23,2,23,3,4,1,0,7.4,7.46,7.40
Linus Ullmark,ID8476999,BOS,G,10793,3,79,76,6,6,64,6,67,6,12,3,0,21.2,7.07,7.07
Joseph Woll,ID8479361,TOR,G,9982,3,94,90,19,1,70,1,72,21,8,2,0,21.0,7.57,7.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pyotr Kochetkov,ID8481611,CAR,G,9151,3,67,56,8,2,46,3,52,12,0,0,0,-10.8,-4.25,-3.60
Laurent Brossoit,ID8476316,WPG,G,3564,1,26,22,2,0,20,0,22,4,0,0,0,-3.6,-3.64,-3.60
Stuart Skinner,ID8479973,EDM,G,9232,3,59,48,11,2,35,3,41,15,1,0,0,-11.4,-4.45,-3.80


In [41]:
team_names
teamURL = 'https://statsapi.web.nhl.com/api/v1/teams/'
appendix = '/stats'

team_stats = pd.DataFrame()

for index, row in team_names.iterrows():

  fetchURL = teamURL + str(row['id']) + appendix

  # print(fetchURL)

  team = requests.get(fetchURL).json()
  stats = team['stats'][0]['splits'][0]['stat']
  name = team['stats'][0]['splits'][0]['team']['name']

  temp = pd.DataFrame([stats])
  temp['name'] = name

  team_stats = pd.concat([team_stats, temp], ignore_index=True)


team_stats = pd.merge(team_stats, team_names, on='name')
team_stats.set_index('name', inplace=True)

In [42]:

team_stats.columns.to_list()


['gamesPlayed',
 'wins',
 'losses',
 'ot',
 'pts',
 'ptPctg',
 'goalsPerGame',
 'goalsAgainstPerGame',
 'evGGARatio',
 'powerPlayPercentage',
 'powerPlayGoals',
 'powerPlayGoalsAgainst',
 'powerPlayOpportunities',
 'penaltyKillPercentage',
 'shotsPerGame',
 'shotsAllowed',
 'winScoreFirst',
 'winOppScoreFirst',
 'winLeadFirstPer',
 'winLeadSecondPer',
 'winOutshootOpp',
 'winOutshotByOpp',
 'faceOffsTaken',
 'faceOffsWon',
 'faceOffsLost',
 'faceOffWinPercentage',
 'shootingPctg',
 'savePctg',
 'abbreviation',
 'id']

In [43]:
for index, row in skaters_summary.iterrows():
  team = row['team']
  gr = 82 - team_stats.loc[team_stats['abbreviation'] == team]['gamesPlayed']

  skaters_summary.loc[index, 'gamesRemain'] = int(gr)

  skaters_summary.loc[index, 'fantasyPace'] = row['FPPG'] * int(gr) + int(row['fantasyPoints'])


skaters_summary['gamesRemain'] = skaters_summary['gamesRemain'].astype(int)

In [44]:
skaters_summary.loc[skaters_summary.index == 'Sebastian Aho']

goodAho = skaters_summary.loc[(skaters_summary.index == 'Sebastian Aho') & (skaters_summary['team'] == 'CAR')]
goodAhoGR = 82 - team_stats.loc[team_stats['abbreviation'] == 'CAR']['gamesPlayed']
goodAhoPace = goodAho['FPPG'] * int(gr) + int(goodAho['fantasyPoints'])

skaters_summary.loc[(skaters_summary.index == 'Sebastian Aho') & (skaters_summary['team'] == 'CAR'), 'fantasyPace'] = int(goodAhoPace)
skaters_summary.loc[(skaters_summary.index == 'Sebastian Aho') & (skaters_summary['team'] == 'CAR'), 'gamesRemain'] = int(goodAhoGR)

badAho = skaters_summary.loc[(skaters_summary.index == 'Sebastian Aho') & (skaters_summary['team'] == 'NYI')]
badAhoGR = 82 - team_stats.loc[team_stats['abbreviation'] == 'NYI']['gamesPlayed']
badAhoPace = badAho['FPPG'] * int(gr) + int(badAho['fantasyPoints'])

skaters_summary.loc[(skaters_summary.index == 'Sebastian Aho') & (skaters_summary['team'] == 'NYI'), 'fantasyPace'] = int(badAhoPace)
skaters_summary.loc[(skaters_summary.index == 'Sebastian Aho') & (skaters_summary['team'] == 'NYI'), 'gamesRemain'] = int(badAhoGR)

In [45]:
skaters_summary.loc[skaters_summary.index == 'Sebastian Aho']

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,blocked,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,fantasyPoints,FPP60,FPPG,gamesRemain,fantasyPace
name,,,,,,,,,,,,,,,,,,,
Sebastian Aho,ID8478427,CAR,C,5045,4,1,2,3,11,1,0,3409,1179,457,6.7,4.78,1.67,75,132.0
Sebastian Aho,ID8480222,NYI,D,4300,5,0,0,0,5,2,7,3622,511,167,4.2,3.52,0.84,77,67.0


In [46]:
for index, row in team_stats.iterrows():
  team_stats.at[index, 'creaseMins'] = goalies_summary.loc[goalies_summary['team'] == row['abbreviation']]['timeOnIce'].sum()

team_stats['creaseMins'] = team_stats['creaseMins'].astype(int)

In [47]:
for index, row in goalies_summary.iterrows():
  team = row['team']
  gr = 82 - team_stats.loc[team_stats['abbreviation'] == team]['gamesPlayed']

  goalies_summary.at[index, 'gamesRemain'] = int(gr)
  cs = row['timeOnIce'] / team_stats.loc[team_stats['abbreviation'] == team]['creaseMins'].sum()


  goalies_summary.at[index, 'fantasyPace'] = (float(row['FPP60']) * int(gr) * cs) + int(row['fantasyPoints'])
  goalies_summary.at[index, 'creaseShare'] = cs * 100

# goalies_summary['gamesRemain'] = goalies_summary['gamesRemain'].astype(int)
# goalies_summary['creaseShare'] = round(goalies_summary['creaseShare'].astype(int), 2)

In [48]:
goalies_summary['gamesRemain'] = goalies_summary['gamesRemain'].astype(int)
goalies_summary['fantasyPace'] = round(goalies_summary['fantasyPace'], 2)
goalies_summary['creaseShare'] = round(goalies_summary['creaseShare'], 2)

goalies_summary

,id,team,position,timeOnIce,gamePlayed,shots,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG,gamesRemain,fantasyPace,creaseShare
name,,,,,,,,,,,,,,,,,,,,,,
James Reimer,ID8473503,DET,G,7200,2,54,52,10,1,41,1,43,10,8,2,1,17.4,8.70,8.70,75,202.52,28.43
Jeremy Swayman,ID8480280,BOS,G,10730,3,93,89,23,4,62,4,65,24,12,3,1,24.8,8.32,8.27,76,339.23,49.85
Anthony Stolarz,ID8476932,FLA,G,3569,1,28,27,2,2,23,2,23,3,4,1,0,7.4,7.46,7.40,76,102.30,16.81
Linus Ullmark,ID8476999,BOS,G,10793,3,79,76,6,6,64,6,67,6,12,3,0,21.2,7.07,7.07,76,290.45,50.15
Joseph Woll,ID8479361,TOR,G,9982,3,94,90,19,1,70,1,72,21,8,2,0,21.0,7.57,7.00,76,287.03,46.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pyotr Kochetkov,ID8481611,CAR,G,9151,3,67,56,8,2,46,3,52,12,0,0,0,-10.8,-4.25,-3.60,75,-125.98,36.39
Laurent Brossoit,ID8476316,WPG,G,3564,1,26,22,2,0,20,0,22,4,0,0,0,-3.6,-3.64,-3.60,76,-48.54,16.46
Stuart Skinner,ID8479973,EDM,G,9232,3,59,48,11,2,35,3,41,15,1,0,0,-11.4,-4.45,-3.80,76,-155.29,42.66


In [49]:
last_season = pd.read_csv('2022-23.csv')
season_prior = pd.read_csv('2021-22.csv')
two_years_ago = pd.read_csv('2020-21.csv')


last_season['date'] = pd.to_datetime(last_season['date'])
season_prior['date'] = pd.to_datetime(season_prior['date'])
two_years_ago['date'] = pd.to_datetime(two_years_ago['date'])

last_season['birthdate'] = pd.to_datetime(last_season['birthdate'])
season_prior['birthdate'] = pd.to_datetime(season_prior['birthdate'])
two_years_ago['birthdate'] = pd.to_datetime(two_years_ago['birthdate'])

In [50]:
for index, row in last_season.iterrows():
    w = 0

    if row['timeOnIce'] > 0:
      last_season.at[index, 'gamePlayed'] = 1


    if row['position'] == "G":
        if row['decision'] == "W":
            d = 4
            w += 1
        elif row['decision'] == "OTL":
            d = 1
        elif row['decision'] == "":
            d = 0
        else:
            d = 0
        last_season.at[index, 'decisionPoints'] = d
        last_season.at[index, 'wins'] = w
    else:
        st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
        last_season.at[index, 'specialTeams'] = st

for index, row in season_prior.iterrows():
    w = 0

    if row['timeOnIce'] > 0:
      season_prior.at[index, 'gamePlayed'] = 1


    if row['position'] == "G":
        if row['decision'] == "W":
            d = 4
            w += 1
        elif row['decision'] == "OTL":
            d = 1
        elif row['decision'] == "":
            d = 0
        else:
            d = 0
        season_prior.at[index, 'decisionPoints'] = d
        season_prior.at[index, 'wins'] = w
    else:
        st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
        season_prior.at[index, 'specialTeams'] = st

for index, row in two_years_ago.iterrows():
    w = 0

    if row['timeOnIce'] > 0:
      two_years_ago.at[index, 'gamePlayed'] = 1


    if row['position'] == "G":
        if row['decision'] == "W":
            d = 4
            w += 1
        elif row['decision'] == "OTL":
            d = 1
        elif row['decision'] == "":
            d = 0
        else:
            d = 0
        two_years_ago.at[index, 'decisionPoints'] = d
        two_years_ago.at[index, 'wins'] = w
    else:
        st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
        two_years_ago.at[index, 'specialTeams'] = st

# Bring in previous seasons

In [51]:
last_three = pd.concat([all_df, last_season, season_prior], axis=0)

last_three = last_three.loc[last_three['timeOnIce'] > 0]

In [52]:
# for index, row in last_season.iterrows():
#     d = 0
#     so = 0
#     w = 0

#     if row['timeOnIce'] > 0:
#       last_season.at[index, 'gamePlayed'] = 1


#     if row['position'] == "G":
#         if (row['shutout'] == 1) & (row['timeOnIce'] < 3446):
#             last_season.at[index, 'shutout'] = 0
#         if row['shutout'] == 1:
#             so = 3
#         else:
#             so = 0
#         if row['decision'] == "W":
#             d = 4
#             w += 1
#         elif row['decision'] == "OTL":
#             d = 1
#         elif row['decision'] == "":
#             d = 0
#         else:
#             d = 0
#         shots_against = row['shortHandedShotsAgainst'] + row['evenShotsAgainst'] + row['powerPlayShotsAgainst']
#         last_season.at[index, 'shots_against'] = shots_against
#         ga = (shots_against - row['saves']) * -2
#         last_season.at[index, 'goals_against'] = abs(ga)/2
#         saves = row['saves'] * .2
#         fp = d + so + ga + saves
#         last_season.at[index, 'fantasyPoints'] = fp
#         last_season.at[index, 'decisionPoints'] = d
#         last_season.at[index, 'wins'] = w
# #         row['fantasyPoints'] = fp
#     else:
#         st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
#         fp = (row['goals']*2) + row['assists'] + (st * .5) + (row['blocked'] * .5) + ((row['hits'] + row['shots']) * .1)
#         last_season.at[index, 'fantasyPoints'] = fp
#         last_season.at[index, 'specialTeams'] = st

In [53]:
# for index, row in season_prior.iterrows():
#     d = 0
#     so = 0
#     w = 0

#     if row['timeOnIce'] > 0:
#       season_prior.at[index, 'gamePlayed'] = 1


#     if row['position'] == "G":
#         if (row['shutout'] == 1) & (row['timeOnIce'] < 3446):
#             season_prior.at[index, 'shutout'] = 0
#         if row['shutout'] == 1:
#             so = 3
#         else:
#             so = 0
#         if row['decision'] == "W":
#             d = 4
#             w += 1
#         elif row['decision'] == "OTL":
#             d = 1
#         elif row['decision'] == "":
#             d = 0
#         else:
#             d = 0
#         shots_against = row['shortHandedShotsAgainst'] + row['evenShotsAgainst'] + row['powerPlayShotsAgainst']
#         season_prior.at[index, 'shots_against'] = shots_against
#         ga = (shots_against - row['saves']) * -2
#         season_prior.at[index, 'goals_against'] = abs(ga)/2
#         saves = row['saves'] * .2
#         fp = d + so + ga + saves
#         season_prior.at[index, 'fantasyPoints'] = fp
#         season_prior.at[index, 'decisionPoints'] = d
#         season_prior.at[index, 'wins'] = w
# #         row['fantasyPoints'] = fp
#     else:
#         st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
#         fp = (row['goals']*2) + row['assists'] + (st * .5) + (row['blocked'] * .5) + ((row['hits'] + row['shots']) * .1)
#         season_prior.at[index, 'fantasyPoints'] = fp
#         season_prior.at[index, 'specialTeams'] = st

In [54]:
# for index, row in two_years_ago.iterrows():
#     d = 0
#     so = 0
#     w = 0

#     if row['timeOnIce'] > 0:
#       two_years_ago.at[index, 'gamePlayed'] = 1


#     if row['position'] == "G":
#         if (row['shutout'] == 1) & (row['timeOnIce'] < 3446):
#             two_years_ago.at[index, 'shutout'] = 0
#         if row['shutout'] == 1:
#             so = 3
#         else:
#             so = 0
#         if row['decision'] == "W":
#             d = 4
#             w += 1
#         elif row['decision'] == "OTL":
#             d = 1
#         elif row['decision'] == "":
#             d = 0
#         else:
#             d = 0
#         shots_against = row['shortHandedShotsAgainst'] + row['evenShotsAgainst'] + row['powerPlayShotsAgainst']
#         two_years_ago.at[index, 'shots_against'] = shots_against
#         ga = (shots_against - row['saves']) * -2
#         two_years_ago.at[index, 'goals_against'] = abs(ga)/2
#         saves = row['saves'] * .2
#         fp = d + so + ga + saves
#         two_years_ago.at[index, 'fantasyPoints'] = fp
#         two_years_ago.at[index, 'decisionPoints'] = d
#         two_years_ago.at[index, 'wins'] = w
# #         row['fantasyPoints'] = fp
#     else:
#         st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
#         fp = (row['goals']*2) + row['assists'] + (st * .5) + (row['blocked'] * .5) + ((row['hits'] + row['shots']) * .1)
#         two_years_ago.at[index, 'fantasyPoints'] = fp
#         two_years_ago.at[index, 'specialTeams'] = st

In [55]:
# last_season.to_csv('f2022-23.csv')
# # season_prior.to_csv('f2021-22.csv')
# two_years_ago.to_csv('f2020-21.csv')

In [56]:
# for index, row in last_three.iterrows():
#     d = 0
#     so = 0
#     w = 0

#     if row['timeOnIce'] > 0:
#       last_three.at[index, 'gamePlayed'] = 1


#     if row['position'] == "G":
#         if (row['shutout'] == 1) & (row['timeOnIce'] < 3446):
#             last_three.at[index, 'shutout'] = 0
#         if row['shutout'] == 1:
#             so = 3
#         else:
#             so = 0
#         if row['decision'] == "W":
#             d = 4
#             w += 1
#         elif row['decision'] == "OTL":
#             d = 1
#         elif row['decision'] == "":
#             d = 0
#         else:
#             d = 0
#         shots_against = row['shortHandedShotsAgainst'] + row['evenShotsAgainst'] + row['powerPlayShotsAgainst']
#         last_three.at[index, 'shots_against'] = shots_against
#         ga = (shots_against - row['saves']) * -2
#         last_three.at[index, 'goals_against'] = abs(ga)/2
#         saves = row['saves'] * .2
#         fp = d + so + ga + saves
#         last_three.at[index, 'fantasyPoints'] = fp
#         last_three.at[index, 'decisionPoints'] = d
#         last_three.at[index, 'wins'] = w
# #         row['fantasyPoints'] = fp
#     else:
#         st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
#         fp = (row['goals']*2) + row['assists'] + (st * .5) + (row['blocked'] * .5) + ((row['hits'] + row['shots']) * .1)
#         last_three.at[index, 'fantasyPoints'] = fp
#         last_three.at[index, 'specialTeams'] = st
# #         row['fantasyPoints'] = fp

In [57]:
# Build summary stats for rankings

def first_non_na_position(x):
    non_na_values = x[x != 'N/A']
    return non_na_values.iloc[0] if not non_na_values.empty else 'N/A'

summary_stats_last_three = last_three.groupby('id').agg({
    'name': 'first',
    'team': 'first',
    'position': first_non_na_position,
    'timeOnIce': 'sum',
    'gamePlayed': 'sum',
    'goals': 'sum',
    'assists': 'sum',
    'specialTeams': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
    'evenTimeOnIce': 'sum',
    'powerPlayTimeOnIce': 'sum',
    'shortHandedTimeOnIce': 'sum',
    'saves': 'sum',
    'powerPlaySaves': 'sum',
    'shortHandedSaves': 'sum',
    'evenSaves': 'sum',
    'shortHandedShotsAgainst': 'sum',
    'evenShotsAgainst': 'sum',
    'powerPlayShotsAgainst': 'sum',
    'decisionPoints': 'sum',
    'wins': 'sum',
    # 'decision': 'sum',
    'shutout': 'sum',
    'fantasyPoints': 'sum'
}).reset_index()

summary_stats_last_three.set_index('name', inplace=True)


# summary_stats = summary_stats.rename(columns={'date': 'gp'})
# summary_stats['gp'] = summary_stats['gp'].apply(lambda x: len(x))
summary_stats_last_three['gamePlayed'] = summary_stats_last_three['gamePlayed'].astype(int)
summary_stats_last_three['decisionPoints'] = summary_stats_last_three['decisionPoints'].astype(int)
summary_stats_last_three['wins'] = summary_stats_last_three['wins'].astype(int)
summary_stats_last_three['specialTeams'] = summary_stats_last_three['specialTeams'].astype(int)


summary_stats_last_three['FPP60'] = (summary_stats_last_three['fantasyPoints'] / summary_stats_last_three['timeOnIce'] * 3600).round(2)
summary_stats_last_three['FPPG'] = (summary_stats_last_three['fantasyPoints'] / summary_stats_last_three['gamePlayed']).round(2)

summary_stats_last_three = summary_stats_last_three.sort_values(by='timeOnIce', ascending=False)
# pd.set_option('display.max_rows', None)

summary_stats_last_three = summary_stats_last_three[summary_stats_last_three['position'] != 'N/A']

summary_stats_last_three.info()

skaters_summary_last_three = summary_stats_last_three.loc[summary_stats_last_three['position'] != "G"]
skaters_summary_last_three = skaters_summary_last_three.drop(columns=['saves', 'powerPlaySaves',	'shortHandedSaves',	'evenSaves',	'shortHandedShotsAgainst',	'evenShotsAgainst',	'powerPlayShotsAgainst',	'decisionPoints',	'wins',	'shutout'])

goalies_summary_last_three = summary_stats_last_three.loc[summary_stats_last_three['position'] == "G"]
goalies_summary_last_three = goalies_summary_last_three.drop(columns=['goals',	'assists',	'specialTeams',	'hits',	'blocked',	'evenTimeOnIce',	'powerPlayTimeOnIce',	'shortHandedTimeOnIce'])


<class 'pandas.core.frame.DataFrame'>
Index: 1300 entries, Juuse Saros to Jett Alexander
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       1300 non-null   object 
 1   team                     1300 non-null   object 
 2   position                 1300 non-null   object 
 3   timeOnIce                1300 non-null   int64  
 4   gamePlayed               1300 non-null   int32  
 5   goals                    1300 non-null   int64  
 6   assists                  1300 non-null   int64  
 7   specialTeams             1300 non-null   int32  
 8   shots                    1300 non-null   int64  
 9   hits                     1300 non-null   int64  
 10  blocked                  1300 non-null   int64  
 11  evenTimeOnIce            1300 non-null   int64  
 12  powerPlayTimeOnIce       1300 non-null   int64  
 13  shortHandedTimeOnIce     1300 non-null   int64  
 14  saves    

In [58]:
skaters_summary_last_three

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,blocked,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,fantasyPoints,FPP60,FPPG
name,,,,,,,,,,,,,,,,,
Brent Burns,ID8470613,CAR,D,252142,171,30,89,45,473,97,245,197796,29423,24923,351.0,5.01,2.05
Rasmus Dahlin,ID8480839,BUF,D,246166,165,28,105,57,392,242,235,202651,32969,10546,370.4,5.42,2.24
Quinn Hughes,ID8480800,VAN,D,244255,160,16,134,72,324,50,132,195618,36341,12296,305.4,4.50,1.91
Victor Hedman,ID8475167,TBL,D,241874,165,30,111,58,421,191,277,191703,30962,19209,399.7,5.95,2.42
Adam Fox,ID8479323,NYR,D,239942,166,24,128,72,324,58,259,186497,32619,20826,379.7,5.70,2.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
John Ludvig,ID8481206,PIT,D,329,1,0,0,0,1,2,0,326,0,3,0.3,3.28,0.30
Mike Vecchione,ID8480087,WSH,RW,306,1,0,0,0,3,0,0,306,0,0,0.3,3.53,0.30
Jeff Malott,ID8482408,WPG,LW,246,1,0,0,0,0,3,0,246,0,0,0.3,4.39,0.30


In [64]:
all_df

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,position,date,name,team,opponent,birthdate,id,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage,shutout,gamePlayed,fantasyPoints,specialTeams,shots_against,goals_against,decisionPoints,wins
ID8478519-10-10-2023,1002,0,0,1,0,0,0,0,60.0,9,15,1,0,0,0,0,-1,690,87,225,C,2023-10-10,Anthony Cirelli,TBL,NSH,1997-07-15,ID8478519,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.1,0.0,NaN,NaN,NaN,NaN
ID8477450-10-10-2023,774,1,1,3,2,0,0,0,40.0,6,15,0,0,0,0,3,1,612,18,144,C,2023-10-10,Jason Dickinson,CHI,PIT,1995-07-04,ID8477450,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,5.0,0.0,NaN,NaN,NaN,NaN
ID8476473-10-10-2023,1078,0,0,0,1,0,0,0,NaN,0,0,0,0,0,0,0,-1,971,18,89,D,2023-10-10,Connor Murphy,CHI,PIT,1993-03-26,ID8476473,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.1,0.0,NaN,NaN,NaN,NaN
ID8475791-10-10-2023,1244,0,0,2,1,0,0,0,NaN,0,0,1,1,0,0,0,0,945,299,0,LW,2023-10-10,Taylor Hall,CHI,PIT,1991-11-14,ID8475791,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.3,0.0,NaN,NaN,NaN,NaN
ID8480252-10-10-2023,555,0,1,2,0,0,0,0,54.55,6,11,0,0,0,0,0,0,531,4,20,C,2023-10-10,Cole Guttman,CHI,PIT,1999-04-06,ID8480252,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.2,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID8476392-24-10-2023,1012,1,0,1,1,0,0,0,25.0,5,20,0,0,0,0,2,0,828,2,182,C,2023-10-24,Adam Lowry,WPG,STL,1993-03-29,ID8476392,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.2,0.0,NaN,NaN,NaN,NaN
ID8480145-24-10-2023,1124,2,0,1,0,0,0,0,NaN,0,0,0,0,0,0,0,2,884,91,149,D,2023-10-24,Neal Pionk,WPG,STL,1995-07-29,ID8480145,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.1,0.0,NaN,NaN,NaN,NaN
ID8480113-24-10-2023,1066,0,0,4,0,0,0,0,NaN,0,0,1,0,0,0,0,0,864,202,0,LW,2023-10-24,Alex Iafallo,WPG,STL,1993-12-21,ID8480113,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.4,0.0,NaN,NaN,NaN,NaN
ID8478891-24-10-2023,912,0,1,2,0,0,0,0,NaN,0,0,1,1,0,0,0,1,799,4,109,C,2023-10-24,Mason Appleton,WPG,STL,1996-01-15,ID8478891,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.2,0.0,NaN,NaN,NaN,NaN


In [59]:
for index, row in skaters_summary.iterrows():
  last_three_fppg = skaters_summary_last_three.loc[skaters_summary_last_three['id'] == row['id']]['FPPG']
  try:
    skaters_summary.at[index, 'last3FPPG'] = last_three_fppg[0]
  except:
    skaters_summary.at[index, 'last3FPPG'] = 0

In [60]:
goalies_summary_last_three
for index, row in goalies_summary.iterrows():
  last_three_fpp60 = goalies_summary_last_three.loc[goalies_summary_last_three['id'] == row['id']]['FPP60']
  try:
    goalies_summary.at[index, 'last3FPPG'] = last_three_fpp60[0]
  except:
    goalies_summary.at[index, 'last3FPPG'] = 0

In [88]:
def convert_time (time):
    time_delta = datetime.timedelta(minutes=time)
    minutes = time_delta.seconds // 60
    seconds = time_delta.seconds % 60
    return f"{minutes:02}:{seconds:02}"
    


topD = skaters_summary.loc[skaters_summary['position'] == 'D'].sort_values('fantasyPace', ascending=False)

# topD['PPT_per_game'] = f"{int(topD['powerPlayTimeOnIce'] // 60)}:{int(topD['powerPlayTimeOnIce'] % 60)}"
    
topD['PPT_per_game'] = ((topD['powerPlayTimeOnIce'] / topD['gamePlayed']) / 60)

for index, row in topD.iterrows():
    topD.at[index, 'QBtime'] = convert_time(topD.at[index, 'PPT_per_game'])
    
topD.sort_values('QBtime', ascending=False)
topD.to_csv('topD.csv', encoding='utf-8')

In [94]:
QBs = topD.loc[topD['PPT_per_game'] > 2]
QB_list = QBs.index.to_list()
QB_list

QB_all = all_df.loc[all_df['name'].isin(QB_list)]
len(QB_list)

51

In [100]:
topD_list = topD.sort_values('fantasyPace', ascending=False)
topD_list = topD_list.head(51)
topD_list = topD_list.index.to_list()
topD_list = set(topD_list) - set(QB_list)
print(list(topD_list))
print(len(list(topD_list)))

['Jake Walman', 'Mikhail Sergachev', 'Scott Mayfield', 'Egor Zamula', 'Dylan DeMelo', 'Artem Zub', 'Alexandre Carrier', 'Jake Middleton', 'Jacob Trouba', 'Justin Barron', 'Kaiden Guhle', 'Brayden McNabb', 'Alec Martinez', 'MacKenzie Weegar', 'TJ Brodie', 'Kaedan Korczak', 'Morgan Rielly', 'Jaccob Slavin', 'Brett Pesce', 'Matt Roy', 'Sean Walker', 'Jonas Brodin', 'Brady Skjei', 'Mario Ferraro', 'Ilya Lyubushkin']
25


In [92]:
opponent_QBs = QB_all.groupby('opponent').agg({
    'timeOnIce': 'sum',
    'assists': 'sum',
    'goals': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
#     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
    'fantasyPoints': 'sum',
    'date': 'nunique',
    'powerPlayGoals': 'sum',
    'powerPlayAssists': 'sum'
    # Add more columns as needed
})

opponent_QBs['FPAPG'] = round(opponent_QBs['fantasyPoints'] / opponent_QBs['date'], 2)
opponent_QBs['FPAPG_Rank'] = opponent_QBs['FPAPG'].rank(ascending=True, method='min').astype(int)

# opponent_stats['BPG'] = round(opponent_stats['blocked'] / opponent_stats['date'], 2)
# opponent_stats['BPG_Rank'] = opponent_stats['BPG'].rank(ascending=True, method='min').astype(int)

opponent_QBs = opponent_QBs.rename_axis('abbreviation')

opponent_QBs = pd.merge(opponent_QBs, team_names, on='abbreviation')

opponent_QBs['powerPlayPoints'] = opponent_QBs['powerPlayGoals'] + opponent_QBs['powerPlayAssists']

opponent_QBs = opponent_QBs.drop(columns=['powerPlayGoals', 'powerPlayAssists'])

opponent_QBs

,abbreviation,timeOnIce,assists,goals,shots,hits,blocked,fantasyPoints,date,FPAPG,FPAPG_Rank,name,id,powerPlayPoints
0,ANA,14651,10,2,21,7,7,24.8,6,4.13,29,Anaheim Ducks,24,9
1,ARI,11508,5,1,11,8,12,16.4,6,2.73,11,Arizona Coyotes,53,3
2,BOS,13550,2,0,16,9,18,13.5,5,2.70,10,Boston Bruins,6,0
3,BUF,13823,4,2,21,2,16,18.8,7,2.69,9,Buffalo Sabres,7,1
4,CAR,16897,8,4,21,8,23,33.9,7,4.84,32,Carolina Hurricanes,12,6
5,CBJ,13834,5,1,20,3,16,18.3,6,3.05,17,Columbus Blue Jackets,29,2
6,CGY,15085,8,1,23,9,18,23.2,7,3.31,21,Calgary Flames,20,2
7,CHI,12535,1,0,23,7,17,12.5,7,1.79,2,Chicago Blackhawks,16,0
8,COL,11644,4,1,14,6,12,15.5,6,2.58,8,Colorado Avalanche,21,3
9,DAL,10114,2,0,19,4,8,8.8,5,1.76,1,Dallas Stars,25,0


In [ ]:
opponent_QBs.loc[opponent_QBs['name'] == game['teams']['home']['team']['name']]['FPAPG'].iloc[0]

In [132]:
url = "https://statsapi.web.nhl.com/api/v1/schedule?startDate=2023-10-26&endDate=2023-11-30"
response = requests.get(url)

# Get the JSON data
data = response.json()

# Create a list to store game information
game_info = []

# Iterate through the dates
for date_info in data['dates']:
        
    games = date_info['games']
    for game in games:
        
        date = game['gameDate']

        datetime_obj = datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ')

        utc_timestamp = pytz.utc.localize(datetime_obj)
        eastern_timezone = pytz.timezone('US/Eastern')
        eastern_time = utc_timestamp.astimezone(eastern_timezone)

        formatted_date_string = eastern_time.strftime('%d-%m-%Y')
        
        game_info.append({
            'Game ID': game['gamePk'],
            'Date': formatted_date_string,
            'Home Team': game['teams']['home']['team']['name'],
            'Home': team_names.loc[team_names['name'] == game['teams']['home']['team']['name']]['abbreviation'].iloc[0],
            'HomeQB': opponent_QBs.loc[opponent_QBs['name'] == game['teams']['home']['team']['name']]['FPAPG'].iloc[0],
            'Away Team': game['teams']['away']['team']['name'],
            'Away': team_names.loc[team_names['name'] == game['teams']['away']['team']['name']]['abbreviation'].iloc[0],
            'AwayQB': opponent_QBs.loc[opponent_QBs['name'] == game['teams']['away']['team']['name']]['FPAPG'].iloc[0],
        })

# Create a DataFrame
df = pd.DataFrame(game_info)

# Display the DataFrame
df = pd.melt(df, id_vars=['Game ID', 'Date', 'HomeQB', 'AwayQB'], value_vars=['Home Team', 'Away Team'], 
             var_name='Location', value_name='Team')

for index, row in df.iterrows():
    if row['Location'] == 'Home Team':
        df.at[index, 'QB_opponent'] = row['AwayQB']
    else:
        df.at[index, 'QB_opponent'] = row['HomeQB']

df.tail(25)

,Game ID,Date,HomeQB,AwayQB,Location,Team,QB_opponent
483,2023020328,28-11-2023,3.83,4.84,Away Team,Carolina Hurricanes,3.83
484,2023020330,28-11-2023,2.77,3.34,Away Team,St. Louis Blues,2.77
485,2023020331,28-11-2023,2.93,2.45,Away Team,Pittsburgh Penguins,2.93
486,2023020332,28-11-2023,3.02,1.76,Away Team,Dallas Stars,3.02
487,2023020333,28-11-2023,1.79,3.10,Away Team,Seattle Kraken,1.79
488,2023020334,28-11-2023,2.73,2.37,Away Team,Tampa Bay Lightning,2.73
489,2023020335,28-11-2023,3.45,3.50,Away Team,Vegas Golden Knights,3.45
490,2023020336,28-11-2023,2.17,4.13,Away Team,Anaheim Ducks,2.17
491,2023020337,29-11-2023,3.05,2.78,Away Team,Montréal Canadiens,3.05
492,2023020338,29-11-2023,3.00,3.20,Away Team,Detroit Red Wings,3.00


In [144]:
QB_forecast = df.groupby('Team')['QB_opponent'].agg(QB_opponent_sum='sum', Team_count='count').reset_index()
QB_forecast['perGame'] = round(QB_forecast['QB_opponent_sum'] / QB_forecast['Team_count'], 2)
# QB_forecast = pd.DataFrame(QB_forecast)
QB_forecast

,Team,QB_opponent_sum,Team_count,perGame
0,Anaheim Ducks,50.66,17,2.98
1,Arizona Coyotes,46.95,16,2.93
2,Boston Bruins,46.95,16,2.93
3,Buffalo Sabres,52.81,16,3.30
4,Calgary Flames,47.28,16,2.96
5,Carolina Hurricanes,46.65,15,3.11
6,Chicago Blackhawks,41.89,14,2.99
7,Colorado Avalanche,47.93,16,3.00
8,Columbus Blue Jackets,54.49,18,3.03
9,Dallas Stars,48.70,16,3.04


In [145]:
last_season

,Unnamed: 0,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,position,date,name,team,opponent,birthdate,id,faceOffPct,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,evenStrengthSavePercentage,shutout,shortHandedSavePercentage,fantasyPoints,shots_against,goals_against,gamePlayed,specialTeams,decisionPoints,wins
0,ID8478508-07-10-2022,1023,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,835,0,188,C,2022-10-07,Yakov Trenin,NSH,SJS,1997-01-13,ID8478508,NaN,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,0,NaN,0.2,NaN,NaN,1.0,0.0,NaN,NaN
1,ID8482133-07-10-2022,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2022-10-07,Thomas Bordeleau,SJS,NSH,2002-01-03,ID8482133,NaN,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN
2,ID8477034-07-10-2022,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2022-10-07,Jaycob Megna,SJS,NSH,1992-12-10,ID8477034,NaN,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN
3,ID8476881-07-10-2022,1261,0,1,2,1,0,0,0,13,25,1,0,0,0,0,0,906,245,110,C,2022-10-07,Tomas Hertl,SJS,NSH,1993-11-12,ID8476881,52.00,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,0,NaN,2.3,NaN,NaN,1.0,0.0,NaN,NaN
4,ID8478067-07-10-2022,558,0,0,2,2,0,0,0,0,0,0,0,0,0,1,-2,430,128,0,LW,2022-10-07,Oskar Lindblom,SJS,NSH,1996-08-15,ID8478067,NaN,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,0,NaN,0.9,NaN,NaN,1.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57699,ID8481528-14-04-2023,955,1,0,2,0,0,0,0,7,17,0,0,0,0,0,1,851,61,43,C,2023-04-14,Dylan Cozens,BUF,CBJ,2001-02-09,ID8481528,41.18,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,0,NaN,1.2,NaN,NaN,1.0,0.0,NaN,NaN
57700,ID8480839-14-04-2023,1570,0,0,3,0,0,0,0,0,0,0,0,0,0,1,0,1433,60,77,D,2023-04-14,Rasmus Dahlin,BUF,CBJ,2000-04-13,ID8480839,NaN,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,0,NaN,0.8,NaN,NaN,1.0,0.0,NaN,NaN
57701,ID8480035-14-04-2023,163,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,163,0,0,D,2023-04-14,Henri Jokiharju,BUF,CBJ,1999-06-17,ID8480035,NaN,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,0,NaN,0.1,NaN,NaN,1.0,0.0,NaN,NaN
57702,ID8477492-14-04-2023,1414,1,3,8,0,0,0,0,6,13,1,1,0,0,0,4,1099,304,11,C,2023-04-14,Nathan MacKinnon,COL,NSH,1995-09-01,ID8477492,46.15,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,0,NaN,7.8,NaN,NaN,1.0,0.0,NaN,NaN


In [62]:
goalies_summary.to_csv('goalies.csv', encoding='utf-8')
skaters_summary.to_csv('skaters.csv', encoding='utf-8')

# Last Season

In [147]:
# Build summary stats for rankings

def first_non_na_position(x):
    non_na_values = x[x != 'N/A']
    return non_na_values.iloc[0] if not non_na_values.empty else 'N/A'

summary_stats_last_season = last_season.groupby('id').agg({
    'name': 'first',
    'team': 'first',
    'position': first_non_na_position,
    'timeOnIce': 'sum',
    'gamePlayed': 'sum',
    'goals': 'sum',
    'assists': 'sum',
    'specialTeams': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
    'evenTimeOnIce': 'sum',
    'powerPlayTimeOnIce': 'sum',
    'shortHandedTimeOnIce': 'sum',
    'saves': 'sum',
    'powerPlaySaves': 'sum',
    'shortHandedSaves': 'sum',
    'evenSaves': 'sum',
    'shortHandedShotsAgainst': 'sum',
    'evenShotsAgainst': 'sum',
    'powerPlayShotsAgainst': 'sum',
    'decisionPoints': 'sum',
    'wins': 'sum',
    # 'decision': 'sum',
    'shutout': 'sum',
    'fantasyPoints': 'sum'
}).reset_index()

summary_stats_last_season.set_index('name', inplace=True)


# summary_stats = summary_stats.rename(columns={'date': 'gp'})
# summary_stats['gp'] = summary_stats['gp'].apply(lambda x: len(x))
summary_stats_last_season['gamePlayed'] = summary_stats_last_season['gamePlayed'].astype(int)
summary_stats_last_season['decisionPoints'] = summary_stats_last_season['decisionPoints'].astype(int)
summary_stats_last_season['wins'] = summary_stats_last_season['wins'].astype(int)
summary_stats_last_season['specialTeams'] = summary_stats_last_season['specialTeams'].astype(int)


summary_stats_last_season['FPP60'] = (summary_stats_last_season['fantasyPoints'] / summary_stats_last_season['timeOnIce'] * 3600).round(2)
summary_stats_last_season['FPPG'] = (summary_stats_last_season['fantasyPoints'] / summary_stats_last_season['gamePlayed']).round(2)

summary_stats_last_season = summary_stats_last_season.sort_values(by='timeOnIce', ascending=False)
# pd.set_option('display.max_rows', None)

summary_stats_last_season = summary_stats_last_season[summary_stats_last_season['position'] != 'N/A']

summary_stats_last_season.info()

last_season_skaters = summary_stats_last_season.loc[summary_stats_last_season['position'] != "G"]
last_season_skaters = last_season_skaters.drop(columns=['saves', 'powerPlaySaves',	'shortHandedSaves',	'evenSaves',	'shortHandedShotsAgainst',	'evenShotsAgainst',	'powerPlayShotsAgainst',	'decisionPoints',	'wins',	'shutout'])

las_season_goalies = summary_stats_last_season.loc[summary_stats_last_season['position'] == "G"]
las_season_goalies = las_season_goalies.drop(columns=['goals',	'assists',	'specialTeams',	'hits',	'blocked',	'evenTimeOnIce',	'powerPlayTimeOnIce',	'shortHandedTimeOnIce'])


<class 'pandas.core.frame.DataFrame'>
Index: 1081 entries, Juuse Saros to Tobie Bisson
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       1081 non-null   object 
 1   team                     1081 non-null   object 
 2   position                 883 non-null    object 
 3   timeOnIce                1081 non-null   int64  
 4   gamePlayed               1081 non-null   int32  
 5   goals                    1081 non-null   int64  
 6   assists                  1081 non-null   int64  
 7   specialTeams             1081 non-null   int32  
 8   shots                    1081 non-null   int64  
 9   hits                     1081 non-null   int64  
 10  blocked                  1081 non-null   int64  
 11  evenTimeOnIce            1081 non-null   int64  
 12  powerPlayTimeOnIce       1081 non-null   int64  
 13  shortHandedTimeOnIce     1081 non-null   int64  
 14  saves      

In [172]:
last_season_D = last_season_skaters.loc[last_season_skaters['position'] == 'D']
last_season_D = last_season_D.sort_values('fantasyPoints', ascending=False)
last_season_D_top_25 = last_season_D.head(25).index.to_list()
last_season_D_top_25

last_season_D_top_10 = last_season_D.head(10).index.to_list()
last_season_D_top_10

['Erik Karlsson',
 'Rasmus Dahlin',
 'Roman Josi',
 'Josh Morrissey',
 'Mikhail Sergachev',
 'Dougie Hamilton',
 'Brandon Montour',
 'Moritz Seider',
 'Alex Pietrangelo',
 'Jacob Trouba']

In [158]:
start_last_season = last_season.loc[last_season['date'] < '2022-10-25']
start_last_season = start_last_season.loc[start_last_season['timeOnIce'] > 0]

In [156]:
all_df.loc[all_df['timeOnIce'] > 0]

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,position,date,name,team,opponent,birthdate,id,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage,shutout,gamePlayed,fantasyPoints,specialTeams,shots_against,goals_against,decisionPoints,wins
ID8478519-10-10-2023,1002,0,0,1,0,0,0,0,60.0,9,15,1,0,0,0,0,-1,690,87,225,C,2023-10-10,Anthony Cirelli,TBL,NSH,1997-07-15,ID8478519,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.1,0.0,NaN,NaN,NaN,NaN
ID8477450-10-10-2023,774,1,1,3,2,0,0,0,40.0,6,15,0,0,0,0,3,1,612,18,144,C,2023-10-10,Jason Dickinson,CHI,PIT,1995-07-04,ID8477450,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,5.0,0.0,NaN,NaN,NaN,NaN
ID8476473-10-10-2023,1078,0,0,0,1,0,0,0,NaN,0,0,0,0,0,0,0,-1,971,18,89,D,2023-10-10,Connor Murphy,CHI,PIT,1993-03-26,ID8476473,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.1,0.0,NaN,NaN,NaN,NaN
ID8475791-10-10-2023,1244,0,0,2,1,0,0,0,NaN,0,0,1,1,0,0,0,0,945,299,0,LW,2023-10-10,Taylor Hall,CHI,PIT,1991-11-14,ID8475791,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.3,0.0,NaN,NaN,NaN,NaN
ID8480252-10-10-2023,555,0,1,2,0,0,0,0,54.55,6,11,0,0,0,0,0,0,531,4,20,C,2023-10-10,Cole Guttman,CHI,PIT,1999-04-06,ID8480252,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.2,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID8476480-24-10-2023,875,0,0,2,0,0,0,0,50.0,3,6,0,0,0,0,0,1,642,110,123,C,2023-10-24,Vladislav Namestnikov,WPG,STL,1992-11-22,ID8476480,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.2,0.0,NaN,NaN,NaN,NaN
ID8476392-24-10-2023,1012,1,0,1,1,0,0,0,25.0,5,20,0,0,0,0,2,0,828,2,182,C,2023-10-24,Adam Lowry,WPG,STL,1993-03-29,ID8476392,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.2,0.0,NaN,NaN,NaN,NaN
ID8480145-24-10-2023,1124,2,0,1,0,0,0,0,NaN,0,0,0,0,0,0,0,2,884,91,149,D,2023-10-24,Neal Pionk,WPG,STL,1995-07-29,ID8480145,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.1,0.0,NaN,NaN,NaN,NaN
ID8480113-24-10-2023,1066,0,0,4,0,0,0,0,NaN,0,0,1,0,0,0,0,0,864,202,0,LW,2023-10-24,Alex Iafallo,WPG,STL,1993-12-21,ID8480113,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.4,0.0,NaN,NaN,NaN,NaN


In [159]:
# Build summary stats for rankings

def first_non_na_position(x):
    non_na_values = x[x != 'N/A']
    return non_na_values.iloc[0] if not non_na_values.empty else 'N/A'

summary_stats_last_season_start = start_last_season.groupby('id').agg({
    'name': 'first',
    'team': 'first',
    'position': first_non_na_position,
    'timeOnIce': 'sum',
    'gamePlayed': 'sum',
    'goals': 'sum',
    'assists': 'sum',
    'specialTeams': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
    'evenTimeOnIce': 'sum',
    'powerPlayTimeOnIce': 'sum',
    'shortHandedTimeOnIce': 'sum',
    'saves': 'sum',
    'powerPlaySaves': 'sum',
    'shortHandedSaves': 'sum',
    'evenSaves': 'sum',
    'shortHandedShotsAgainst': 'sum',
    'evenShotsAgainst': 'sum',
    'powerPlayShotsAgainst': 'sum',
    'decisionPoints': 'sum',
    'wins': 'sum',
    # 'decision': 'sum',
    'shutout': 'sum',
    'fantasyPoints': 'sum'
}).reset_index()

summary_stats_last_season_start.set_index('name', inplace=True)


# summary_stats = summary_stats.rename(columns={'date': 'gp'})
# summary_stats['gp'] = summary_stats['gp'].apply(lambda x: len(x))
summary_stats_last_season_start['gamePlayed'] = summary_stats_last_season_start['gamePlayed'].astype(int)
summary_stats_last_season_start['decisionPoints'] = summary_stats_last_season_start['decisionPoints'].astype(int)
summary_stats_last_season_start['wins'] = summary_stats_last_season_start['wins'].astype(int)
summary_stats_last_season_start['specialTeams'] = summary_stats_last_season_start['specialTeams'].astype(int)


summary_stats_last_season_start['FPP60'] = (summary_stats_last_season_start['fantasyPoints'] / summary_stats_last_season_start['timeOnIce'] * 3600).round(2)
summary_stats_last_season_start['FPPG'] = (summary_stats_last_season_start['fantasyPoints'] / summary_stats_last_season_start['gamePlayed']).round(2)

summary_stats_last_season_start = summary_stats_last_season_start.sort_values(by='timeOnIce', ascending=False)
# pd.set_option('display.max_rows', None)

summary_stats_last_season_start = summary_stats_last_season_start[summary_stats_last_season_start['position'] != 'N/A']

summary_stats_last_season_start.info()

last_season_skaters_start = summary_stats_last_season_start.loc[summary_stats_last_season_start['position'] != "G"]
last_season_skaters_start = last_season_skaters_start.drop(columns=['saves', 'powerPlaySaves',	'shortHandedSaves',	'evenSaves',	'shortHandedShotsAgainst',	'evenShotsAgainst',	'powerPlayShotsAgainst',	'decisionPoints',	'wins',	'shutout'])

las_season_goalies_start = summary_stats_last_season_start.loc[summary_stats_last_season_start['position'] == "G"]
las_season_goalies_start = las_season_goalies_start.drop(columns=['goals',	'assists',	'specialTeams',	'hits',	'blocked',	'evenTimeOnIce',	'powerPlayTimeOnIce',	'shortHandedTimeOnIce'])


<class 'pandas.core.frame.DataFrame'>
Index: 727 entries, Thatcher Demko to Nick Schmaltz
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       727 non-null    object 
 1   team                     727 non-null    object 
 2   position                 727 non-null    object 
 3   timeOnIce                727 non-null    int64  
 4   gamePlayed               727 non-null    int32  
 5   goals                    727 non-null    int64  
 6   assists                  727 non-null    int64  
 7   specialTeams             727 non-null    int32  
 8   shots                    727 non-null    int64  
 9   hits                     727 non-null    int64  
 10  blocked                  727 non-null    int64  
 11  evenTimeOnIce            727 non-null    int64  
 12  powerPlayTimeOnIce       727 non-null    int64  
 13  shortHandedTimeOnIce     727 non-null    int64  
 14  saves   

In [173]:
last_season_D_start = last_season_skaters_start.loc[last_season_skaters_start['position'] == 'D']
last_season_D_start = last_season_D_start.sort_values('fantasyPoints', ascending=False)
last_season_D_start_top_25 = last_season_D_start.head(25).index.to_list()
last_season_D_start_top_25

last_season_D_start_top_10 = last_season_D_start.head(10).index.to_list()
last_season_D_start_top_10

['Rasmus Dahlin',
 'John Carlson',
 'Matt Roy',
 'Ivan Provorov',
 'David Savard',
 'Darnell Nurse',
 'Shayne Gostisbehere',
 'Alec Martinez',
 'Jeff Petry',
 'Zach Werenski']

In [163]:
topD_slow_start = set(last_season_D_top_25) - set(last_season_D_start_top_25)
print(list(topD_start))
print(len(list(topD_start)))

['Moritz Seider', 'Justin Faulk', 'Devon Toews', 'Roman Josi', 'Josh Morrissey', 'Drew Doughty', 'Miro Heiskanen', 'Jared Spurgeon', 'Brandon Montour', 'Dougie Hamilton', 'Rasmus Andersson', 'Brent Burns', 'Charlie McAvoy', 'Quinn Hughes']
14


In [171]:
topD_faded_start = set(last_season_D_start_top_25) - set(last_season_D_top_25)
print(list(topD_faded_start))
print(len(list(topD_faded_start)))

for element in topD_faded_start:
    print(element, end=", ")

['Ivan Provorov', 'Bowen Byram', 'Dmitry Orlov', 'Brayden McNabb', 'Tony DeAngelo', 'John Carlson', 'Matt Roy', 'Zach Werenski', 'Justin Holl', 'Jeff Petry', 'David Savard', 'Noah Dobson', 'Shayne Gostisbehere', 'Neal Pionk']
14
Ivan Provorov, Bowen Byram, Dmitry Orlov, Brayden McNabb, Tony DeAngelo, John Carlson, Matt Roy, Zach Werenski, Justin Holl, Jeff Petry, David Savard, Noah Dobson, Shayne Gostisbehere, Neal Pionk, 

In [165]:
topD_stayed_start = set(last_season_D_start_top_25) & set(last_season_D_top_25)
print(list(topD_stayed_start))
print(len(list(topD_stayed_start)))

['Jacob Trouba', 'Darnell Nurse', 'Alec Martinez', 'Mikhail Sergachev', 'Rasmus Dahlin', 'Adam Fox', 'Alex Pietrangelo', 'Adam Larsson', 'Cale Makar', 'Erik Karlsson', 'Victor Hedman']
11


In [174]:
top10D_stayed_start = set(last_season_D_start_top_10) & set(last_season_D_top_10)
print(list(top10D_stayed_start))
print(len(list(top10D_stayed_start)))

['Rasmus Dahlin']
1


In [178]:
top10D_stayed_start = set(last_season_D_start_top_10) & set(last_season_D_top_25)
print(list(top10D_stayed_start))
print(len(list(top10D_stayed_start)))

['Rasmus Dahlin', 'Darnell Nurse', 'Alec Martinez']
3


In [166]:
dec_last_season = last_season.loc[last_season['date'] < '2022-12-01']
dec_last_season = dec_last_season.loc[dec_last_season['timeOnIce'] > 0]

In [167]:
# Build summary stats for rankings

def first_non_na_position(x):
    non_na_values = x[x != 'N/A']
    return non_na_values.iloc[0] if not non_na_values.empty else 'N/A'

summary_stats_last_season_dec = dec_last_season.groupby('id').agg({
    'name': 'first',
    'team': 'first',
    'position': first_non_na_position,
    'timeOnIce': 'sum',
    'gamePlayed': 'sum',
    'goals': 'sum',
    'assists': 'sum',
    'specialTeams': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
    'evenTimeOnIce': 'sum',
    'powerPlayTimeOnIce': 'sum',
    'shortHandedTimeOnIce': 'sum',
    'saves': 'sum',
    'powerPlaySaves': 'sum',
    'shortHandedSaves': 'sum',
    'evenSaves': 'sum',
    'shortHandedShotsAgainst': 'sum',
    'evenShotsAgainst': 'sum',
    'powerPlayShotsAgainst': 'sum',
    'decisionPoints': 'sum',
    'wins': 'sum',
    # 'decision': 'sum',
    'shutout': 'sum',
    'fantasyPoints': 'sum'
}).reset_index()

summary_stats_last_season_dec.set_index('name', inplace=True)


# summary_stats = summary_stats.rename(columns={'date': 'gp'})
# summary_stats['gp'] = summary_stats['gp'].apply(lambda x: len(x))
summary_stats_last_season_dec['gamePlayed'] = summary_stats_last_season_dec['gamePlayed'].astype(int)
summary_stats_last_season_dec['decisionPoints'] = summary_stats_last_season_dec['decisionPoints'].astype(int)
summary_stats_last_season_dec['wins'] = summary_stats_last_season_dec['wins'].astype(int)
summary_stats_last_season_dec['specialTeams'] = summary_stats_last_season_dec['specialTeams'].astype(int)


summary_stats_last_season_dec['FPP60'] = (summary_stats_last_season_dec['fantasyPoints'] / summary_stats_last_season_dec['timeOnIce'] * 3600).round(2)
summary_stats_last_season_dec['FPPG'] = (summary_stats_last_season_dec['fantasyPoints'] / summary_stats_last_season_dec['gamePlayed']).round(2)

summary_stats_last_season_dec = summary_stats_last_season_dec.sort_values(by='timeOnIce', ascending=False)
# pd.set_option('display.max_rows', None)

summary_stats_last_season_dec = summary_stats_last_season_dec[summary_stats_last_season_dec['position'] != 'N/A']

summary_stats_last_season_dec.info()

last_season_skaters_dec = summary_stats_last_season_dec.loc[summary_stats_last_season_dec['position'] != "G"]
last_season_skaters_dec = last_season_skaters_dec.drop(columns=['saves', 'powerPlaySaves',	'shortHandedSaves',	'evenSaves',	'shortHandedShotsAgainst',	'evenShotsAgainst',	'powerPlayShotsAgainst',	'decisionPoints',	'wins',	'shutout'])

las_season_goalies_dec = summary_stats_last_season_dec.loc[summary_stats_last_season_dec['position'] == "G"]
las_season_goalies_dec = las_season_goalies_dec.drop(columns=['goals',	'assists',	'specialTeams',	'hits',	'blocked',	'evenTimeOnIce',	'powerPlayTimeOnIce',	'shortHandedTimeOnIce'])


<class 'pandas.core.frame.DataFrame'>
Index: 842 entries, Darcy Kuemper to Shane Bowers
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       842 non-null    object 
 1   team                     842 non-null    object 
 2   position                 842 non-null    object 
 3   timeOnIce                842 non-null    int64  
 4   gamePlayed               842 non-null    int32  
 5   goals                    842 non-null    int64  
 6   assists                  842 non-null    int64  
 7   specialTeams             842 non-null    int32  
 8   shots                    842 non-null    int64  
 9   hits                     842 non-null    int64  
 10  blocked                  842 non-null    int64  
 11  evenTimeOnIce            842 non-null    int64  
 12  powerPlayTimeOnIce       842 non-null    int64  
 13  shortHandedTimeOnIce     842 non-null    int64  
 14  saves     

In [176]:
last_season_D_dec = last_season_skaters_dec.loc[last_season_skaters_dec['position'] == 'D']
last_season_D_dec = last_season_D_dec.sort_values('fantasyPoints', ascending=False)
last_season_D_dec_top_25 = last_season_D_dec.head(25).index.to_list()
last_season_D_dec_top_25

last_season_D_dec_top_10 = last_season_D_dec.head(10).index.to_list()
last_season_D_dec_top_25

['Rasmus Dahlin',
 'Erik Karlsson',
 'Mikhail Sergachev',
 'Roman Josi',
 'Alex Pietrangelo',
 'Alec Martinez',
 'Adam Fox',
 'Cale Makar',
 'Noah Dobson',
 'Josh Morrissey',
 'Jacob Trouba',
 'Brandon Montour',
 'Ivan Provorov',
 'John Carlson',
 'Moritz Seider',
 'Justin Faulk',
 'Dougie Hamilton',
 'Filip Hronek',
 'Darnell Nurse',
 'Jeff Petry',
 'Sean Durzi',
 'Rasmus Andersson',
 'Hampus Lindholm',
 'David Savard',
 'Matt Roy']

In [182]:
topD_stayed_dec = set(last_season_D_dec_top_25) & set(last_season_D_top_25)
print(list(topD_stayed_dec))
print(len(list(topD_stayed_dec)))

for element in topD_stayed_dec:
    print(element, end=", ")

['Moritz Seider', 'Justin Faulk', 'Jacob Trouba', 'Darnell Nurse', 'Alec Martinez', 'Mikhail Sergachev', 'Roman Josi', 'Josh Morrissey', 'Rasmus Dahlin', 'Brandon Montour', 'Dougie Hamilton', 'Rasmus Andersson', 'Alex Pietrangelo', 'Adam Fox', 'Cale Makar', 'Erik Karlsson']
16
Moritz Seider, Justin Faulk, Jacob Trouba, Darnell Nurse, Alec Martinez, Mikhail Sergachev, Roman Josi, Josh Morrissey, Rasmus Dahlin, Brandon Montour, Dougie Hamilton, Rasmus Andersson, Alex Pietrangelo, Adam Fox, Cale Makar, Erik Karlsson, 

In [183]:
top10D_stayed_dec = set(last_season_D_dec_top_10) & set(last_season_D_top_10)
print(list(top10D_stayed_dec))
print(len(list(top10D_stayed_dec)))

for element in top10D_stayed_dec:
    print(element, end=", ")

['Roman Josi', 'Mikhail Sergachev', 'Josh Morrissey', 'Rasmus Dahlin', 'Alex Pietrangelo', 'Erik Karlsson']
6
Roman Josi, Mikhail Sergachev, Josh Morrissey, Rasmus Dahlin, Alex Pietrangelo, Erik Karlsson, 

In [184]:
top10D_stayed_dec = set(last_season_D_dec_top_10) & set(last_season_D_top_25)
print(list(top10D_stayed_dec))
print(len(list(top10D_stayed_dec)))

for element in top10D_stayed_dec:
    print(element, end=", ")

['Alec Martinez', 'Roman Josi', 'Mikhail Sergachev', 'Josh Morrissey', 'Rasmus Dahlin', 'Adam Fox', 'Alex Pietrangelo', 'Cale Makar', 'Erik Karlsson']
9
Alec Martinez, Roman Josi, Mikhail Sergachev, Josh Morrissey, Rasmus Dahlin, Adam Fox, Alex Pietrangelo, Cale Makar, Erik Karlsson, 

In [186]:
topD_slow_start = set(last_season_D_top_25) - set(last_season_D_dec_top_25)
print(list(topD_slow_start))
print(len(list(topD_slow_start)))

['Devon Toews', 'Jared Spurgeon', 'Drew Doughty', 'Miro Heiskanen', 'Adam Larsson', 'Brent Burns', 'Charlie McAvoy', 'Victor Hedman', 'Quinn Hughes']
9


In [193]:
# this_season_D = topD.loc[all_df['position'] == 'D']
this_season_D = topD.sort_values('fantasyPoints', ascending=False)
this_season_D_top_25 = this_season_D.head(25).index.to_list()


this_season_D_top_10 = this_season_D.head(10).index.to_list()
this_season_D_top_25

for element in this_season_D_top_25:
    print(element, end=", ")

Cale Makar, Moritz Seider, Jaccob Slavin, Shayne Gostisbehere, Evan Bouchard, Noah Dobson, Shea Theodore, Jacob Trouba, Rasmus Dahlin, Victor Hedman, Jake Sanderson, Alexandre Carrier, Ivan Provorov, Jakob Chychrun, Brayden McNabb, Mike Matheson, Dougie Hamilton, Jake Walman, MacKenzie Weegar, John Carlson, Brady Skjei, Josh Morrissey, Dylan DeMelo, Mikhail Sergachev, Jake Middleton, 

In [192]:
topD_stayed_arrive = set(last_season_D_dec_top_25) - set(last_season_D_start_top_25)
topDarrived = set(topD_stayed_arrive) & set(last_season_D_top_25)
print(list(topDarrived))
print(len(list(topDarrived)))

for element in topDarrived:
    print(element, end=", ")

['Moritz Seider', 'Justin Faulk', 'Roman Josi', 'Josh Morrissey', 'Brandon Montour', 'Dougie Hamilton', 'Rasmus Andersson']
7
Moritz Seider, Justin Faulk, Roman Josi, Josh Morrissey, Brandon Montour, Dougie Hamilton, Rasmus Andersson, 

In [204]:
topPPtime_last_season = last_season_D.sort_values('powerPlayTimeOnIce', ascending=False)
topPPtime_last_season = topPPtime_last_season.head(25)
topPPtime_last_season = topPPtime_last_season.index.to_list()

topBS_last_season = last_season_D.sort_values('blocked', ascending=False)
topBS_last_season = topBS_last_season.head(25)
topBS_last_season = topBS_last_season.index.to_list()

topD_PPTime = set(topPPtime_last_season) & set(last_season_D_top_25)

for element in topD_PPTime:
    print(element, end=", ")
print(len(list(topD_PPTime)))

Moritz Seider, Roman Josi, Josh Morrissey, Drew Doughty, Miro Heiskanen, Rasmus Dahlin, Brandon Montour, Dougie Hamilton, Rasmus Andersson, Adam Fox, Brent Burns, Cale Makar, Erik Karlsson, Quinn Hughes, 14


In [205]:
topD_PPTime = set(topPPtime_last_season) - set(last_season_D_top_25)

for element in topD_PPTime:
    print(element, end=", ")
print(len(list(topD_PPTime)))

Calen Addison, Thomas Chabot, Noah Dobson, Vince Dunn, Seth Jones, Cam Fowler, Aaron Ekblad, Tyson Barrie, Tony DeAngelo, Shayne Gostisbehere, Kris Letang, 11


In [214]:

print('Defense in the top 25 fantasy, but not leading in PP or blocks')
topDCheck_last_season = set(last_season_D_top_25) - set(topPPtime_last_season).union(set(topBS_last_season))
for element in topDCheck_last_season:
    print(element, end=", ")
print(len(list(topDCheck_last_season)))
print('')

print('Defense in the top 25 fantasy and in both top 25 PP and blocks')
topDCheck_last_season = set(last_season_D_top_25) & set(topPPtime_last_season) & set(topBS_last_season)
for element in topDCheck_last_season:
    print(element, end=", ")
print(len(list(topDCheck_last_season)))
print('')



Defense in the top 25 fantasy, but not leading in PP or blocks
Charlie McAvoy, Justin Faulk, Devon Toews, Victor Hedman, 4

Defense in the top 25 fantasy and in both top 25 PP and blocks
Moritz Seider, 1



In [232]:

topFP = topD.sort_values('fantasyPoints', ascending=False)
topFP = topFP.head(25)
topFP = topFP.index.to_list()

topBS = topD.sort_values('blocked', ascending=False)
topBS = topBS.head(25)
topBS = topBS.index.to_list()

topPP = topD.sort_values('powerPlayTimeOnIce', ascending=False)
topPP = topPP.head(25)
topPP = topPP.index.to_list()



topDCheck = set(topFP) - set(topPP).union(set(topBS))
print('Defense in the top 25 fantasy, but not leading in PP or blocks')
for element in topDCheck:
    print(element, end=", ")
print(len(list(topDCheck)))
print('')

topDCheck = set(topPP) - set(topFP)
print('Defense in the top 25 for PP, but not in fantasy')
for element in topDCheck:
    print(element, end=", ")
print(len(list(topDCheck)))
print('')

topDCheck = set(topBS) - set(topFP)
print('Defense in the top 25 for blocks, but not in fantasy')
for element in topDCheck:
    print(element, end=", ")
print(len(list(topDCheck)))
print('')

topDCheck = set(topFP) & set(topPP) & (set(topBS))
print('Defense in the top 25 for all three')
for element in topDCheck:
    print(element, end=", ")
print(len(list(topDCheck)))
print('')

topDCheck = set(topFP) & set(topPP).union(set(topBS))
print('Defense in the top 25 fantasy and at least one of the other factors')
for element in topDCheck:
    print(f"[b]<b>{element}, D, {team_names.loc[team_names['abbreviation'] == (topD.loc[topD.index == element]['team'].iloc[0])]['name'].iloc[0]}: </b>")
    print('')
print(len(list(topDCheck)))
print('')



Defense in the top 25 fantasy, but not leading in PP or blocks
Jakob Chychrun, Dougie Hamilton, Brady Skjei, Dylan DeMelo, Jaccob Slavin, Jake Middleton, 6

Defense in the top 25 for PP, but not in fantasy
Calen Addison, Dmitry Orlov, Tyson Barrie, Oliver Ekman-Larsson, Roman Josi, Drew Doughty, John Klingberg, Vince Dunn, Seth Jones, Brent Burns, Charlie McAvoy, Tony DeAngelo, Thomas Chabot, Sean Durzi, 14

Defense in the top 25 for blocks, but not in fantasy
Niko Mikkola, Connor Murphy, Ryan McDonagh, Matt Roy, TJ Brodie, Jonas Brodin, Seth Jones, Ilya Lyubushkin, Adam Larsson, Mark Giordano, Mario Ferraro, Erik Gudbranson, Brenden Dillon, Jamie Oleksiak, 14

Defense in the top 25 for all three
Moritz Seider, John Carlson, Mike Matheson, 3

Defense in the top 25 fantasy and at least one of the other factors
[b]<b>Ivan Provorov, D, Columbus Blue Jackets: </b>

[b]<b>Jake Walman, D, Detroit Red Wings: </b>

[b]<b>Mikhail Sergachev, D, Tampa Bay Lightning: </b>

[b]<b>Rasmus Dahlin, D, 

In [229]:
team_names

,abbreviation,name,id
0,NJD,New Jersey Devils,1
1,NYI,New York Islanders,2
2,NYR,New York Rangers,3
3,PHI,Philadelphia Flyers,4
4,PIT,Pittsburgh Penguins,5
5,BOS,Boston Bruins,6
6,BUF,Buffalo Sabres,7
7,MTL,Montréal Canadiens,8
8,OTT,Ottawa Senators,9
9,TOR,Toronto Maple Leafs,10


# Last Two

In [172]:
last_two = pd.concat([all_df, last_season], axis=0)

last_two = last_two.loc[last_two['timeOnIce'] > 0]

In [173]:
last_two = last_two.loc[last_two['date'] > '2022-12-31']
last_two

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,position,date,name,team,opponent,birthdate,id,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage,shutout,gamePlayed,fantasyPoints,specialTeams,shots_against,goals_against,decisionPoints,wins,Unnamed: 0
ID8478519-10-10-2023,1002,0,0,1,0,0,0,0,60.0,9,15,1,0,0,0,0,-1,690,87,225,C,2023-10-10,Anthony Cirelli,TBL,NSH,1997-07-15,ID8478519,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.1,0.0,NaN,NaN,NaN,NaN,NaN
ID8477450-10-10-2023,774,1,1,3,2,0,0,0,40.0,6,15,0,0,0,0,3,1,612,18,144,C,2023-10-10,Jason Dickinson,CHI,PIT,1995-07-04,ID8477450,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN
ID8476473-10-10-2023,1078,0,0,0,1,0,0,0,NaN,0,0,0,0,0,0,0,-1,971,18,89,D,2023-10-10,Connor Murphy,CHI,PIT,1993-03-26,ID8476473,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.1,0.0,NaN,NaN,NaN,NaN,NaN
ID8475791-10-10-2023,1244,0,0,2,1,0,0,0,NaN,0,0,1,1,0,0,0,0,945,299,0,LW,2023-10-10,Taylor Hall,CHI,PIT,1991-11-14,ID8475791,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.3,0.0,NaN,NaN,NaN,NaN,NaN
ID8480252-10-10-2023,555,0,1,2,0,0,0,0,54.55,6,11,0,0,0,0,0,0,531,4,20,C,2023-10-10,Cole Guttman,CHI,PIT,1999-04-06,ID8480252,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.2,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57698,876,0,0,1,0,0,0,0,66.67,2,3,1,0,0,0,1,-1,850,0,26,LW,2023-04-14,Zemgus Girgensons,BUF,CBJ,1994-01-05,ID8476878,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.6,0.0,NaN,NaN,NaN,NaN,ID8476878-14-04-2023
57699,955,1,0,2,0,0,0,0,41.18,7,17,0,0,0,0,0,1,851,61,43,C,2023-04-14,Dylan Cozens,BUF,CBJ,2001-02-09,ID8481528,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,1.2,0.0,NaN,NaN,NaN,NaN,ID8481528-14-04-2023
57700,1570,0,0,3,0,0,0,0,NaN,0,0,0,0,0,0,1,0,1433,60,77,D,2023-04-14,Rasmus Dahlin,BUF,CBJ,2000-04-13,ID8480839,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.8,0.0,NaN,NaN,NaN,NaN,ID8480839-14-04-2023
57701,163,0,0,1,0,0,0,0,NaN,0,0,0,0,0,0,0,-1,163,0,0,D,2023-04-14,Henri Jokiharju,BUF,CBJ,1999-06-17,ID8480035,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.1,0.0,NaN,NaN,NaN,NaN,ID8480035-14-04-2023


In [174]:
# Build summary stats for rankings

def first_non_na_position(x):
    non_na_values = x[x != 'N/A']
    return non_na_values.iloc[0] if not non_na_values.empty else 'N/A'

summary_stats_last_two = last_two.groupby('id').agg({
    'name': 'first',
    'team': 'first',
    'position': first_non_na_position,
    'timeOnIce': 'sum',
    'gamePlayed': 'sum',
    'goals': 'sum',
    'assists': 'sum',
    'specialTeams': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
    'evenTimeOnIce': 'sum',
    'powerPlayTimeOnIce': 'sum',
    'shortHandedTimeOnIce': 'sum',
    'saves': 'sum',
    'powerPlaySaves': 'sum',
    'shortHandedSaves': 'sum',
    'evenSaves': 'sum',
    'shortHandedShotsAgainst': 'sum',
    'evenShotsAgainst': 'sum',
    'powerPlayShotsAgainst': 'sum',
    'decisionPoints': 'sum',
    'wins': 'sum',
    # 'decision': 'sum',
    'shutout': 'sum',
    'fantasyPoints': 'sum'
}).reset_index()

summary_stats_last_two.set_index('name', inplace=True)


# summary_stats = summary_stats.rename(columns={'date': 'gp'})
# summary_stats['gp'] = summary_stats['gp'].apply(lambda x: len(x))
summary_stats_last_two['gamePlayed'] = summary_stats_last_two['gamePlayed'].astype(int)
summary_stats_last_two['decisionPoints'] = summary_stats_last_two['decisionPoints'].astype(int)
summary_stats_last_two['wins'] = summary_stats_last_two['wins'].astype(int)
summary_stats_last_two['specialTeams'] = summary_stats_last_two['specialTeams'].astype(int)


summary_stats_last_two['FPP60'] = (summary_stats_last_two['fantasyPoints'] / summary_stats_last_two['timeOnIce'] * 3600).round(2)
summary_stats_last_two['FPPG'] = (summary_stats_last_two['fantasyPoints'] / summary_stats_last_two['gamePlayed']).round(2)

summary_stats_last_two = summary_stats_last_two.sort_values(by='timeOnIce', ascending=False)
# pd.set_option('display.max_rows', None)

summary_stats_last_two = summary_stats_last_two[summary_stats_last_two['position'] != 'N/A']

summary_stats_last_two.info()

skaters_summary_last_two = summary_stats_last_two.loc[summary_stats_last_two['position'] != "G"]
skaters_summary_last_two = skaters_summary_last_two.drop(columns=['saves', 'powerPlaySaves',	'shortHandedSaves',	'evenSaves',	'shortHandedShotsAgainst',	'evenShotsAgainst',	'powerPlayShotsAgainst',	'decisionPoints',	'wins',	'shutout'])

goalies_summary_last_two = summary_stats_last_two.loc[summary_stats_last_two['position'] == "G"]
goalies_summary_last_two = goalies_summary_last_two.drop(columns=['goals',	'assists',	'specialTeams',	'hits',	'blocked',	'evenTimeOnIce',	'powerPlayTimeOnIce',	'shortHandedTimeOnIce'])


<class 'pandas.core.frame.DataFrame'>
Index: 1011 entries, Juuse Saros to Ryan Carpenter
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       1011 non-null   object 
 1   team                     1011 non-null   object 
 2   position                 1011 non-null   object 
 3   timeOnIce                1011 non-null   int64  
 4   gamePlayed               1011 non-null   int32  
 5   goals                    1011 non-null   int64  
 6   assists                  1011 non-null   int64  
 7   specialTeams             1011 non-null   int32  
 8   shots                    1011 non-null   int64  
 9   hits                     1011 non-null   int64  
 10  blocked                  1011 non-null   int64  
 11  evenTimeOnIce            1011 non-null   int64  
 12  powerPlayTimeOnIce       1011 non-null   int64  
 13  shortHandedTimeOnIce     1011 non-null   int64  
 14  saves    

In [185]:
skaters_summary_last_two.loc[skaters_summary_last_two.index == 'Nazem Kadri']

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,blocked,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,fantasyPoints,FPP60,FPPG
name,,,,,,,,,,,,,,,,,
Nazem Kadri,ID8475172,CGY,C,50726,50,10,18,6,163,57,12,42248,8090,388,69.0,4.9,1.38


In [176]:
skaters_summary.loc[(skaters_summary['team'] == 'CHI') & (skaters_summary['position'] == 'RW')]

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,blocked,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,fantasyPoints,FPP60,FPPG,gamesRemain,fantasyPace,last3FPPG
name,,,,,,,,,,,,,,,,,,,,
Corey Perry,ID8470621,CHI,RW,5757,6,2,3,0,9,6,3,3989,1768,0,10.0,6.25,1.67,76,136.92,1.07
Taylor Raddysh,ID8479390,CHI,RW,5939,6,1,1,1,13,7,7,4417,924,598,9.0,5.46,1.50,76,123.00,1.14
MacKenzie Entwistle,ID8480025,CHI,RW,1120,2,1,0,0,2,3,0,1062,14,44,2.5,8.04,1.25,76,97.00,0.76


In [177]:
goalies_summary.head(25)

,id,team,position,timeOnIce,gamePlayed,shots,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG,gamesRemain,fantasyPace,creaseShare,last3FPPG
name,,,,,,,,,,,,,,,,,,,,,,,
James Reimer,ID8473503,DET,G,7200,2,54,52,10,1,41,1,43,10,8,2,1,17.4,8.70,8.70,76,239.16,33.60,1.36
Eric Comrie,ID8477480,BUF,G,3600,1,25,24,5,0,19,0,20,5,4,1,0,6.8,6.80,6.80,77,111.01,20.06,2.00
Alex Nedeljkovic,ID8477968,PIT,G,3582,1,36,34,3,0,31,0,32,4,4,1,0,6.8,6.83,6.80,77,112.56,20.26,1.20
Alexandar Georgiev,ID8480382,COL,G,18252,5,140,132,29,7,96,7,103,30,20,5,1,33.4,6.59,6.68,77,540.43,100.00,3.15
Jordan Binnington,ID8476412,STL,G,11390,3,98,94,7,2,85,2,88,8,9,2,0,19.8,6.26,6.60,78,390.29,76.04,1.44
Jeremy Swayman,ID8480280,BOS,G,7141,2,70,66,21,4,41,4,44,22,8,2,0,13.2,6.65,6.60,78,219.54,39.82,3.54
Joseph Woll,ID8479361,TOR,G,6409,2,59,56,6,1,49,1,51,7,4,1,0,12.2,6.85,6.10,77,199.66,35.58,4.91
Karel Vejmelka,ID8478872,ARI,G,10900,3,103,98,19,4,75,4,76,23,8,2,0,17.6,5.81,5.87,77,287.70,60.51,0.91
Logan Thompson,ID8480313,VGK,G,7199,2,62,58,2,1,55,1,58,3,8,2,0,11.6,5.80,5.80,76,156.81,33.08,3.09


In [178]:
skaters_summary

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,blocked,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,fantasyPoints,FPP60,FPPG,gamesRemain,fantasyPace,last3FPPG
name,,,,,,,,,,,,,,,,,,,,
Jack Hughes,ID8481559,NJD,C,5506,4,4,6,7,22,1,0,4400,1082,24,19.8,12.95,4.95,78,405.10,2.60
Jesper Bratt,ID8479407,NJD,LW,4722,4,3,5,7,10,8,2,3521,1069,132,17.3,13.19,4.32,78,353.96,1.94
Sam Reinhart,ID8477933,FLA,C,6397,5,6,2,3,19,0,8,4246,1384,767,21.4,12.04,4.28,77,350.56,2.17
Alex DeBrincat,ID8479337,DET,RW,6167,6,8,4,4,19,3,2,5156,1008,3,25.2,14.71,4.20,76,344.20,2.22
Auston Matthews,ID8479318,TOR,C,7181,5,6,0,2,30,12,7,5455,1144,582,20.7,10.38,4.14,77,338.78,3.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pavel Buchnevich,ID8477402,STL,LW,1608,2,0,0,0,0,1,0,1182,281,145,0.1,0.22,0.05,78,3.90,2.06
Chris Tierney,ID8476919,NJD,C,455,1,0,0,0,0,0,0,455,0,0,0.0,0.00,0.00,78,0.00,0.75
Liam Foudy,ID8480853,CBJ,C,476,1,0,0,0,0,0,0,476,0,0,0.0,0.00,0.00,77,0.00,0.67


In [187]:
eastern = pytz.timezone('US/Eastern')

for day in days:
    date = day['date']
    games = day['games']

    day_of_week = datetime.strptime(date, '%Y-%m-%d').strftime('%A').upper()
    print(day_of_week)

    for game in games:
        home_team = game['teams']['home']['team']['name']
        away_team = game['teams']['away']['team']['name']
        venue = game['venue']['name']
        game_time_utc = datetime.fromisoformat(game['gameDate'][:-1]).replace(tzinfo=pytz.utc)
        game_time_et = game_time_utc.astimezone(eastern)

        formatted_time = game_time_et.strftime('%I:%M %p').lstrip('0').replace(':00', '').replace('PM', 'p.m.')

        home_abbr = opponent_stats.loc[opponent_stats['name'] == home_team, 'abbreviation'].values[0]
        home_top5 = top_players_by_team.loc[top_players_by_team['team'] == home_abbr]

        away_abbr = opponent_stats.loc[opponent_stats['name'] == away_team, 'abbreviation'].values[0]
        away_top5 = top_players_by_team.loc[top_players_by_team['team'] == away_abbr]

#         print(f"On {date}, {away_team} at {home_team}")
        print(f"\033[1m{away_team} at {home_team}\033[0m")
        print(f"{formatted_time}, {venue}")
        print()
        print(f"<p><b>Puck Line: </b>Team </p>")
        print(f"<p><b>Moneyline: </b>{away_team} (-110), {home_team} (-110)")
        print(f"<p><b>Total: </b> 5</p>")
        print()
        print(f"<p><b>Best bet: </b></p>")
        print(f"<p>")
        print(f"<p><b>{away_team} betting trends: </b> </p>")
        print(f"<p><b>{home_team} betting trends: </b> </p>")
              
        
        
#         print(f"{away_abbr}: {opponent_stats.loc[opponent_stats['name'] == away_team, 'FPAPG'].values[0]} FPAPG, Rank: {opponent_stats.loc[opponent_stats['name'] == away_team, 'FPAPG_Rank'].values[0]}")
#         print(f"\033[1mTop Players in FPPG\033[0m")
#         for index, row in away_top5.iterrows():
#             print(f"{index}, FPPG: {row['FPPG']}, FP: {round(row['fantasyPoints'], 2)}")
#         print()
#         print(f"{home_abbr}: {opponent_stats.loc[opponent_stats['name'] == home_team, 'FPAPG'].values[0]} FPAPG, Rank: {opponent_stats.loc[opponent_stats['name'] == home_team, 'FPAPG_Rank'].values[0]}")
#         print(f"\033[1mTop Players in FPPG\033[0m")
#         for index, row in home_top5.iterrows():
#             print(f"{index}, FPPG: {row['FPPG']}, FP: {round(row['fantasyPoints'], 2)}")

        print()
#         print("\033[1mThis is bold text\033[0m")


MONDAY
Montréal Canadiens at Buffalo Sabres
7 p.m., KeyBank Center

<p><b>Puck Line: </b>Team </p>
<p><b>Moneyline: </b>Montréal Canadiens (-110), Buffalo Sabres (-110)
<p><b>Total: </b> 5</p>

<p><b>Best bet: </b></p>
<p>
<p><b>Montréal Canadiens betting trends: </b> </p>
<p><b>Buffalo Sabres betting trends: </b> </p>

TUESDAY
Toronto Maple Leafs at Washington Capitals
6 p.m., Capital One Arena

<p><b>Puck Line: </b>Team </p>
<p><b>Moneyline: </b>Toronto Maple Leafs (-110), Washington Capitals (-110)
<p><b>Total: </b> 5</p>

<p><b>Best bet: </b></p>
<p>
<p><b>Toronto Maple Leafs betting trends: </b> </p>
<p><b>Washington Capitals betting trends: </b> </p>

Anaheim Ducks at Columbus Blue Jackets
6:30 p.m., Nationwide Arena

<p><b>Puck Line: </b>Team </p>
<p><b>Moneyline: </b>Anaheim Ducks (-110), Columbus Blue Jackets (-110)
<p><b>Total: </b> 5</p>

<p><b>Best bet: </b></p>
<p>
<p><b>Anaheim Ducks betting trends: </b> </p>
<p><b>Columbus Blue Jackets betting trends: </b> </p>

Buffalo 

In [247]:
check = last_season.loc[(last_season['team'] == 'NJD') & (last_season['opponent'] == 'MTL')]
check.groupby('name').sum()

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,faceOffPct,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,savePercentage,powerPlaySavePercentage,evenStrengthSavePercentage,shutout,shortHandedSavePercentage,fantasyPoints,shots_against,goals_against,gamePlayed,specialTeams,decisionPoints,wins
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Akira Schmid,3600,0,0,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,23,0,0,23,0,24,0,95.833333,0.0,95.833333,0,0.0,6.6,24.0,1.0,1.0,0.0,4.0,1.0
Alexander Holtz,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Brendan Smith,3004,0,0,8,2,0,0,4,0,0,1,1,0,0,5,1,2951,0,53,0.00,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0.0,3.5,0.0,0.0,3.0,0.0,0.0,0.0
Curtis Lazar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Damon Severson,3369,1,0,3,1,0,0,2,0,0,1,3,0,0,4,-1,3272,97,0,0.00,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0.0,3.4,0.0,0.0,3.0,0.0,0.0,0.0
Dawson Mercer,2880,3,1,8,0,1,0,2,0,0,2,0,0,0,3,1,2717,100,63,0.00,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0.0,7.8,0.0,0.0,3.0,1.0,0.0,0.0
Dougie Hamilton,3674,2,1,9,3,0,1,0,0,0,2,6,0,0,6,2,3233,376,65,0.00,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0.0,8.7,0.0,0.0,3.0,1.0,0.0,0.0
Erik Haula,2688,0,0,4,1,0,0,0,17,32,2,1,0,0,2,0,2416,82,190,165.48,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0.0,1.5,0.0,0.0,3.0,0.0,0.0,0.0
Fabian Zetterlund,1611,0,0,3,1,0,0,0,1,1,0,0,0,0,0,1,1590,21,0,100.00,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0.0,0.4,0.0,0.0,2.0,0.0,0.0,0.0


In [244]:
# all_df.loc[(all_df['position'] == 'D') & (all_df['opponent'] == 'OTT') & (all_df['goals'] > 0)]
all_df.loc[all_df['name'] == '']

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,position,date,name,team,opponent,birthdate,id,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage,shutout,gamePlayed,fantasyPoints,specialTeams,shots_against,goals_against,decisionPoints,wins
ID8477426-10-10-2023,972,1,2,3,0,2,0,0,27.27,3,11,1,2,0,0,1,1,531,390,51,LW,2023-10-10,Nicholas Paul,TBL,NSH,1995-03-20,ID8477426,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,6.8,2.0,NaN,NaN,NaN,NaN
ID8477426-14-10-2023,1203,0,0,2,2,0,0,0,73.33,11,15,0,0,0,0,1,-2,976,168,59,LW,2023-10-14,Nicholas Paul,TBL,DET,1995-03-20,ID8477426,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.9,0.0,NaN,NaN,NaN,NaN
ID8477426-15-10-2023,747,0,0,3,2,0,0,0,50.0,4,8,0,0,0,0,1,-3,625,122,0,LW,2023-10-15,Nicholas Paul,TBL,OTT,1995-03-20,ID8477426,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,1.0,0.0,NaN,NaN,NaN,NaN
ID8477426-17-10-2023,938,0,0,0,3,0,0,0,20.0,1,5,0,0,0,0,0,0,597,274,67,LW,2023-10-17,Nicholas Paul,TBL,BUF,1995-03-20,ID8477426,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.3,0.0,NaN,NaN,NaN,NaN
ID8477426-19-10-2023,1009,1,1,2,0,1,1,0,55.56,5,9,1,2,0,0,0,0,844,165,0,LW,2023-10-19,Nicholas Paul,TBL,VAN,1995-03-20,ID8477426,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,4.2,2.0,NaN,NaN,NaN,NaN
ID8477426-21-10-2023,967,0,0,1,0,0,0,0,60.0,9,15,3,1,0,0,0,0,765,186,16,LW,2023-10-21,Nicholas Paul,TBL,TOR,1995-03-20,ID8477426,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.1,0.0,NaN,NaN,NaN,NaN


In [248]:
# opponent_stats['BPG'] = round(opponent_stats['blocked'] / opponent_stats['date'], 2)
# opponent_stats['BPG_Rank'] = opponent_stats['BPG'].rank(ascending=True, method='min').astype(int)

opponent_stats['PPG'] = round((opponent_stats['assists'] + opponent_stats['goals']) / opponent_stats['date'], 2)
opponent_stats['PPG_Rank'] = opponent_stats['PPG'].rank(ascending=True, method='min').astype(int)



opponent_stats

,abbreviation,timeOnIce,assists,goals,shots,hits,blocked,fantasyPoints,date,BPG,BPG_Rank,name,id,powerPlayPoints,PPPG,PPPG_Rank,PPG,PPG_Rank
0,ANA,87797,29,15,150,78,69,126.8,5,13.80,4,Anaheim Ducks,24,21,4.20,31,8.80,18
1,ARI,88265,17,9,157,114,72,105.6,5,14.40,7,Arizona Coyotes,53,15,3.00,21,5.20,4
2,BOS,88628,13,7,150,97,83,94.7,5,16.60,19,Boston Bruins,6,3,0.60,5,4.00,1
3,BUF,88616,26,15,146,114,88,129.0,5,17.60,27,Buffalo Sabres,7,3,0.60,5,8.20,14
4,CAR,105692,48,30,165,90,123,210.0,6,20.50,32,Carolina Hurricanes,12,24,4.00,30,13.00,32
5,CBJ,89127,26,16,172,56,83,127.8,5,16.60,19,Columbus Blue Jackets,29,9,1.80,14,8.40,15
6,CGY,106818,37,22,175,78,96,155.8,6,16.00,15,Calgary Flames,20,3,0.50,2,9.83,25
7,CHI,105288,33,18,211,100,96,155.1,6,16.00,15,Chicago Blackhawks,16,9,1.50,11,8.50,17
8,COL,89559,13,8,140,79,78,91.4,5,15.60,14,Colorado Avalanche,21,3,0.60,5,4.20,2
9,DAL,73036,15,9,127,102,65,91.9,4,16.25,17,Dallas Stars,25,0,0.00,1,6.00,5
